## Step 1 Generate PepPI Interaction

In [1]:
import pandas as pd
import numpy as np
import os


In [ ]:
import pandas as pd
import numpy as np
import os

def check_abnormal_aa(peptide_seq):
    len_seq = len(peptide_seq)
    cnt = 0
    standard_aa = ['G','A','P','V','L','I','M','F','Y','W','S','T','C','N','Q','K','H','R','D','E']
    for i in peptide_seq:
        if i in standard_aa :
            cnt = cnt+1
    score = float(cnt)/len_seq
    return score

def lower_chain(input_str):
    chain_list = list(input_str)
    output_list = []

    for item in chain_list:
        if item.isalpha() :
            a=item.lower()
        else :
            a=item
        output_list.append(a)
    output_str = ''.join(output_list)
    return output_str


# Step 0: parse the fasta file downloaded from the RCSB PDB
# INPUT : pdb_seqres.txt
# OUTPUT: pdb_pep_chain, pdbid_all_fasta
raw_str=''
print("Processing pdb_seqres")
with open('E:/example_data_feature/data_prepare/pdb_seqres2.txt','r') as f:
    for line in f.readlines():
        raw_str = raw_str+line.replace('\n','###')
raw_list = raw_str.split('>')
del raw_list[0]

PDB_id_lst = [x.split('_')[0] for x in raw_list]
PDB_chain_lst = [x.split('_')[1].split(' ')[0].lower() for x in raw_list]
PDB_type_lst = [x.split('mol:')[1].split(' ')[0] for x in raw_list]
PDB_seq_lst = [x.split('###')[1] for x in raw_list]
PDB_seq_len_lst = [len(x) for x in PDB_seq_lst]
df_fasta_raw =pd.DataFrame(list(zip(PDB_type_lst, PDB_seq_len_lst,PDB_seq_lst,PDB_id_lst,PDB_chain_lst)),\
                       columns=['PDB_type','PDB_seq_len','PDB_seq','PDB_id','chain'])
df_fasta = df_fasta_raw[(df_fasta_raw.PDB_seq_len<=50)&(df_fasta_raw.PDB_type=='protein')]
df_fasta_raw.to_csv('pdbid_all_fasta_all', encoding='utf-8', index=False, sep = '\t')
df_fasta.to_csv('pdb_pep_chain_all', encoding='utf-8', index=False, sep = '\t')

print('Step 0 is finished by generating two files : pdb_pep_chain & pdbid_all_fasta!')
# Step1 : Load all PDB ids that might contain peptide interaction and plip prediction results
# INPUT : pdb_pep_chain from Step 0 & analyzed file generated by PLIP (placed under ./peptide_result/ ,details in Data_construction.txt)
# OUTPUT: plip_predict_result

def load(pdb_pep_dataset,plip_result_filename): #pdb_pep_chain   #plip_predict_result
	df_fasta_pep = pd.read_csv(pdb_pep_dataset,sep='\t',header = 0) 
	df_fasta_pep=df_fasta_pep.reset_index(drop=True)
	df_predict = pd.DataFrame(columns=['pdb_id','pep_chain','predicted_chain'])

	for i in range(df_fasta_pep.shape[0]):
		pdb_id = df_fasta_pep['PDB_id'][i]
		chain = df_fasta_pep['chain'][i]
		print("PDB_ID:", pdb_id)
# 		os.system(f'python D:/Anaconda3/Lib/site-packages/plip-2.2.2-py3.8.egg/plip/plipcmd.py -i {pdb_id} --peptide {chain.upper()}  -vto E:/plip/outputresults/ --name {pdb_id}_{chain}_result')
		result_file_name = 'E:/plip/outputresults/'+pdb_id + '_'+chain+'_result.txt'
	    # print(result_file_name)
		try:
			for line in open(result_file_name):
				if line.startswith('Interacting chain(s):'):
					df_predict.loc[i] = [pdb_id,chain, str(line).replace('\n','')\
	                                     .replace('\r','')\
	                                     .replace('Interacting chain(s):','')
	                                     .lower()]
			if i % 5000 == 0:
				print('already finished files',i)
		except :
	        #print('find no file for',pdb_id)
	        #print(i,pdb_id,line)
			pass
	print('finish loading!')
	print('-----------------------------------------------------')
	print(df_predict)
	df_predict['predicted_chain_num'] = df_predict.predicted_chain.apply(lambda x :len(x.replace(' ','')) )
	df_predict = df_predict.loc[df_predict.predicted_chain_num>0]
	df_predict = df_predict.drop('predicted_chain_num',axis = 1)
	df_predict['predicted_chain'] = df_predict.predicted_chain.apply(lambda x :\
	                                      x.replace(' ',''))
	df_predict['pep_chain'] = df_predict.pep_chain.apply(lambda x :\
	                                      x.replace(' ',''))
	df_predict = df_predict.reset_index(drop=True)
	print('finish removing PDB ids without any interaction')
	print('-----------------------------------------------------')
	df_predict.predicted_chain = df_predict.predicted_chain.apply(lambda x: x.split(','))
	lst_col = 'predicted_chain'
	df1 = pd.DataFrame({col:np.repeat(df_predict[col].values, df_predict[lst_col].str.len())
	              for col in df_predict.columns.difference([lst_col])
	            }).assign(**{lst_col:np.concatenate(df_predict[lst_col].values)})[df_predict.columns.tolist()]
	df_predict = df1
	# save organized data formatted like (pdb,pep_chain,predicted_prot_chain)
	file_name = plip_result_filename
	df_predict.to_csv(file_name, encoding='utf-8', index=False, sep='\t')
	print('finish exploding comma-seperated predicted chain, successfully saved records:',df_predict.shape[0])

	print('Step 1 is finished by generating the PLIP prediction file : plip_predict_result. ')

	return df_predict

# Step 2: Get fasta sequence of the predicted interacting chains
# INPUT:  pdbid_all_fasta from Step 0
# OUTPUT: -
def load_all_fasta(all_fasta_file,input_dataset): # pdbid_all_fasta # df_predict
	df_fasta = pd.read_csv(all_fasta_file,sep = '\t', header = 0)
	df_fasta_protein = df_fasta.loc[df_fasta.PDB_type=='protein']

	# df_fasta_protein['PDB_id'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[0])
	# df_fasta_protein['chain'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[1].lower())
	df_fasta_vocabulary = df_fasta_protein[['PDB_id','chain','PDB_seq']]

	df_predict_det = pd.merge(input_dataset,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','pep_chain'],right_on = ['PDB_id','chain'])

	df_predict_det1 = pd.merge(df_predict_det,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','predicted_chain'],right_on = ['PDB_id','chain'])
	df_predict_det1 =df_predict_det1.drop(['PDB_id_x','chain_x','PDB_id_y','chain_y'],axis =1)
	df_predict_det1.columns = ['pdb_id','pep_chain','predicted_chain','pep_seq','prot_seq']
	print(df_predict_det1)    
	df_predict_det1['pep_seq_len'] = df_predict_det1.pep_seq.apply(lambda x: len(x))
	df_predict_det1['prot_seq_len'] = df_predict_det1.prot_seq.apply(lambda x: len(str(x)))
    

	# check sequence length(peptide<=50 & protein >50)
	df_predict_det1 = df_predict_det1.loc[(df_predict_det1.pep_seq_len <= 50) & (df_predict_det1.prot_seq_len > 50)]


	# remove records with more than 20% AA is abnormal
	df_predict_det1['peptide_seq_score'] = df_predict_det1.pep_seq.apply(lambda x: check_abnormal_aa(x))
	df_predict_det1 = df_predict_det1[df_predict_det1.peptide_seq_score >= 0.8]

	print('finish removing sequences without too many non-standard residues')
	print('-----------------------------------------------------')


	return df_predict_det1

# Step 3: Map Uniprot ID for PDB complex by protein-chain & PDB id
# INPUT: data from Step 2 & pdb_chain_uniprot.tsv from SIFT
# OUTPUT: UniProt_ID_list ( all IDs are the searching query on https://www.uniprot.org/uploadlists/ for unified sequence)
def map_uniprot_chain(input_dataset,pdb_chain_uniprot_file): #df_predict_det1 #pdb_chain_uniprot.tsv
	df_sifts = pd.read_csv(pdb_chain_uniprot_file, sep = '\t', header = 0)
	df_sifts = df_sifts[['PDB','CHAIN','SP_PRIMARY']]
	df_sifts_keep = df_sifts[df_sifts['CHAIN'] !=  df_sifts['CHAIN']]
	df_sifts = df_sifts[df_sifts['CHAIN'] == df_sifts['CHAIN']]
	df_sifts['CHAIN'] = df_sifts.CHAIN.apply(lambda x: lower_chain(x))

	df_predict_det2 = pd.merge(input_dataset,df_sifts, how = 'left', \
                              left_on = ['pdb_id','predicted_chain'],right_on = ['PDB','CHAIN'])
	df_predict_det2 = df_predict_det2.drop(['PDB','CHAIN'],axis = 1)


	# subset records that don't have a matched protein chain Uniprot
	df_predict_det2_no_uni = df_predict_det2[df_predict_det2.SP_PRIMARY != df_predict_det2.SP_PRIMARY]
	df_predict_det2_no_uni = df_predict_det2_no_uni.reset_index(drop = True)
	
	df_predict_det2_no_uni = df_predict_det2_no_uni.drop(['prot_seq_len','peptide_seq_score'],axis = 1)
	df_predict_det2_no_uni = df_predict_det2_no_uni[['pdb_id','pep_chain','predicted_chain','pep_seq',\
	                                 'pep_seq_len','SP_PRIMARY','prot_seq']]
	df_predict_det2_no_uni.rename(columns = {'prot_seq': 'Sequence'}, inplace=True)

	# focus on records with Uniprot Ids
	df_predict_det3 = df_predict_det2[df_predict_det2.SP_PRIMARY == df_predict_det2.SP_PRIMARY]

	# save matched uniprot ID for retrieving information from Uniprot Website
	df_uni_id = df_predict_det3[['SP_PRIMARY']]
	df_uni_id.drop_duplicates(inplace = True)
	file_name = pdb_chain_uniprot_file
	df_uni_id.to_csv('pdb_chain_uniprot_all.tsv', encoding = 'utf-8', index = False, sep = '\t')

	return df_predict_det2_no_uni,df_predict_det3


# Step 4: Load Uniport sequences and family information & filter out MHC families
# INPUT: the data from Step 3 & uniprot2seq from UniProt Website (a tab separated file with fields including Uniprot_id,Uniprot Sequence,Protein_name,Protein_families)
# OUTPUT: interacted peptide-protein pairs from PDB (a '#' separated file with fields including pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families)

def load_uni_seq(df_predict_det2_no_uni, input_dataset,uniprot2seq_file):

	df_uni2seq = pd.read_csv(uniprot2seq_file,sep = '\t',header = 0)
	# df_uni2seq = df_uni2seq.drop('uniprot',axis = 1)
	df_uni2seq = df_uni2seq.drop_duplicates(['Uniprot_id','Sequence'],keep = 'first')
	df_uni2seq = df_uni2seq.fillna('Unknown_from_uniprot')

	# join by uniprot id
	df_predict_det4 = pd.merge(input_dataset,df_uni2seq,how = 'left',left_on = ['SP_PRIMARY'],right_on = ['Uniprot_id'])
	df_predict_det4 = df_predict_det4.drop(['Uniprot_id','Protein_name','prot_seq','prot_seq_len','peptide_seq_score'],axis = 1)
	df_predict_det4 = df_predict_det4.drop_duplicates(['pdb_id','pep_seq','SP_PRIMARY','Sequence'],keep = 'first')
	print(df_predict_det4)

	# filter out MHC
	df_predict_det4["MHC_flag"] = df_predict_det4.Protein_families.apply(lambda x: x.lower().find('mhc'))
	df_mhc = df_predict_det4.loc[df_predict_det4.MHC_flag!=-1][["pdb_id","Protein_families"]]
	df_mhc.columns = ['pdb_id_mhc','prot_family_mhc']

	# join by  PDB id only(if a pdb contains mhc proteins,remove all records of the PDB id)
	df_predict_det5 = pd.merge(df_predict_det4, df_mhc,left_on = ['pdb_id'], right_on = ['pdb_id_mhc'], how='left')
	df_predict_det5 = df_predict_det5.loc[df_predict_det5.pdb_id_mhc!=df_predict_det5.pdb_id_mhc]
	df_predict_det5 = df_predict_det5.drop(['pdb_id_mhc','prot_family_mhc','MHC_flag'],axis =1)
	df_predict_det5.drop_duplicates(inplace=True)
	print(df_predict_det5)
	print(df_predict_det2_no_uni)
	if(len(df_predict_det2_no_uni)>0):
# 		df_predict_det2_no_uni = df_predict_det2_no_uni.drop(['prot_seq_len','peptide_seq_score'],axis = 1)
		df_predict_det2_no_uni = df_predict_det2_no_uni[['pdb_id','pep_chain','predicted_chain','pep_seq','pep_seq_len','SP_PRIMARY','Sequence']]
# 		df_predict_det2_no_uni.rename(columns={'prot_seq': 'Sequence'}, inplace=True)
		df_predict_det2_no_uni['Protein_families'] = pd.Series(['Unknown Uniprot_ids' for x in range(df_predict_det2_no_uni.shape[0])])
	df_predict_det6 = pd.concat([df_predict_det2_no_uni,df_predict_det5],ignore_index=True)
	df_predict_det6['plip_prot_chain'] = df_predict_det6.predicted_chain.apply(lambda x:\
	                                                              x.upper())
	df_predict_det6 = df_predict_det6.drop_duplicates(['pep_seq','Sequence'],keep='first')
	df_predict_det6 = df_predict_det6.reset_index(drop=True) #8184
	df_predict_det6['prot_seq_len'] = df_predict_det6.Sequence.apply(lambda x: len(str(x)))
	df_predict_det6 = df_predict_det6[df_predict_det6.prot_seq_len<=5000]

	df_pdb_pairs = df_predict_det6[['pdb_id','pep_chain','plip_prot_chain','pep_seq','SP_PRIMARY','Sequence','Protein_families']]
	df_pdb_pairs.columns = ['pdb_id','pep_chain','prot_chain','pep_seq','Uniprot_id','prot_seq','protein_families']
	file_name = 'train_pairs_pdb'
	df_pdb_pairs.to_csv(file_name, encoding = 'utf-8', index = False)
	print(df_pdb_pairs)

	return df_pdb_pairs


df_predict = load('pdb_pep_chain_all','plip_predict_result_all')
df_predict_det1 = load_all_fasta('pdbid_all_fasta_all',df_predict)
df_predict_det2_no_uni,df_predict_det3 = map_uniprot_chain(df_predict_det1,'E:/example_data_feature/data_prepare/pdb_chain_uniprot.tsv')
df_pdb_pairs = load_uni_seq(df_predict_det2_no_uni, df_predict_det3,'uniprot2seq2.tsv')


In [4]:
pep_pairs = pd.read_csv('E:/Projek pak Wisnu/Projek Peptide-Protein Interaction/train_pairs_pdb_modified.csv')
pep_pairs

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families,pdb_chain,Uniprot_chain
0,1acy,p,L,YNKRKRIHIGPGRAFYTTKNIIGC,Unknown0,DIVMTQSPASLVVSLGQRATISCRASESVDSYGKSFMHWYQQKPGQ...,Unknown Uniprot_ids,1acy_p,Unknown0_L
1,1qgc,5,A,TTAYTASARGDLAHLTTTAARTLP,Unknown1,EVMLVESGGGLVKPGGSLKLSCTASGFIFNRCAMSWVRQTPEKRLE...,Unknown Uniprot_ids,1qgc_5,Unknown1_A
2,1vrk,b,A,RRKWQKTGHAVRAIGRLSSSX,Unknown2,ADQLTDEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEA...,Unknown Uniprot_ids,1vrk_b,Unknown2_A
3,1xcq,p,A,STNPKPQRKTKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLG,Unknown3,DIVMSQSPSSLAVSAGEKVTMSCKSSQSLLNSRTRKNYLAWYQQKP...,Unknown Uniprot_ids,1xcq_p,Unknown3_A
4,1xcq,p,B,STNPKPQRKTKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLG,Unknown4,QIQLVQSGPELKKPGETVKISCKASGYTFTDFSMHWVNQAPGKGLN...,Unknown Uniprot_ids,1xcq_p,Unknown4_B
...,...,...,...,...,...,...,...,...,...
8493,7wvw,l,R,MYFINILTL,P25090,METNFSTPLNEYEEVSYESAGYTVLRILPLVVLGVTFVLGVLGNGL...,G-protein coupled receptor 1 family,7wvw_l,P25090_R
8494,7yxw,d,A,PPSNPPPRPPAEARKKPS,P14598,MGDTFIRHIALLGFEKRFVPSQHYVYMFLVKWQDLSEKVVYRRFTE...,Unknown_from_uniprot,7yxw_d,P14598_A
8495,7zez,d,B,ARTKQTARKSTGG,Q03164,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,Class V-like SAM-binding methyltransferase sup...,7zez_d,Q03164_B
8496,8gch,c,F,GAW,P00766,CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSL...,Peptidase S1 family,8gch_c,P00766_F


In [2]:
pep_pairs2 = pd.read_csv('E:/Projek pak Wisnu/Projek Peptide-Protein Interaction/train_pairs_cancer+drugbank.csv')
pep_pairs2

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families,pdb_chain,Uniprot_chain
0,1dev,b,A,SQSPNPNNPAEYCSTIPPLQQAQASGALSSPPPTVMVPVGV,Q15796,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,Dwarfin/SMAD family,1dev_b,Q15796_A
1,1fm6,e,D,CPSSHSSLTERHKILHRLLQEGSPS,P37231,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"Nuclear hormone receptor family, NR1 subfamily",1fm6_e,P37231_D
2,1icf,b,I,NNKYWLVKNSWGEEWGMGGYVKMAKDRRNHCGIASAASYPTV,P04233,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,Unknown_from_uniprot,1icf_b,P04233_I
3,1jdh,b,A,LGANDELISFKDEGEQEEKSSENSSAERDLADVKSSLV,P35222,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Beta-catenin family,1jdh_b,P35222_A
4,1jmt,b,A,KKKVRKYWDVPPPGFEHITPMQYKAMQA,Q01081,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,Splicing factor SR family,1jmt_b,Q01081_A
...,...,...,...,...,...,...,...,...,...
447,NaN,NaN,NaN,TKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC,Q12879,MGRVGYWTLLVLPALLVWRGPAPSAAAEKGPPALNIAVMLGHSHDV...,NaN,DB00142,Q12879
448,NaN,NaN,NaN,TKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC,Q12879,MGRVGYWTLLVLPALLVWRGPAPSAAAEKGPPALNIAVMLGHSHDV...,NaN,DB00145,Q12879
449,NaN,NaN,NaN,TKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC,Q13315,MSLVLNDLLICCRQLEHDRATERKKEVEKFKRLIRDPETIKHLDRH...,NaN,DB00201,Q13315
450,NaN,NaN,NaN,TKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC,Q15596,MSGMGENTSDPSRAETRKRKECPDQLGPSPKRNTEKRNREQENKYI...,NaN,DB00255,Q15596


In [5]:
sum = 0
for i in pep_pairs2['pep_seq']:
    sum = sum + len(i)
print(sum/ len(pep_pairs2['pep_seq']))


sum2 = 0
for i in pep_pairs2['pep_seq']:
    if(len(i)<18):
        sum2 = sum2+1
print(sum2)

18.9070796460177
277


In [8]:
pep_pairs[pep_pairs['Uniprot_id'].isin(prot_cancer['Uniprot_Accession'])]

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families,pdb_chain,Uniprot_chain
1175,1dev,b,A,SQSPNPNNPAEYCSTIPPLQQAQASGALSSPPPTVMVPVGV,Q15796,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,Dwarfin/SMAD family,1dev_b,Q15796_A
1209,1fm6,e,D,CPSSHSSLTERHKILHRLLQEGSPS,P37231,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"Nuclear hormone receptor family, NR1 subfamily",1fm6_e,P37231_D
1238,1icf,b,I,NNKYWLVKNSWGEEWGMGGYVKMAKDRRNHCGIASAASYPTV,P04233,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,Unknown_from_uniprot,1icf_b,P04233_I
1262,1jdh,b,A,LGANDELISFKDEGEQEEKSSENSSAERDLADVKSSLV,P35222,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Beta-catenin family,1jdh_b,P35222_A
1272,1jmt,b,A,KKKVRKYWDVPPPGFEHITPMQYKAMQA,Q01081,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,Splicing factor SR family,1jmt_b,Q01081_A
...,...,...,...,...,...,...,...,...,...
8442,7rns,b,A,KDSYVGDEA,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,PI3K p85 subunit family,7rns_b,P27986_A
8445,7rov,e,A,AXPLYISYDPVCRA,P01116,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,"Small GTPase superfamily, Ras family",7rov_e,P01116_A
8463,7tbh,a,J,LSRRPSYRKILNDLSSDAP,Q92793,MAENLLDGPPNPKRAKLSSPGFSANDSTDFGSLFDLENDLPDELIP...,Unknown_from_uniprot,7tbh_a,Q92793_J
8489,7wgo,b,A,LTERHKILHRLLQEG,P37231,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"Nuclear hormone receptor family, NR1 subfamily",7wgo_b,P37231_A


In [3]:
pep_pairs2 = pd.read_csv('E:/Projek pak Wisnu/Projek Peptide-Protein Interaction/train_pairs_cancer+drugbank.csv')
pep_pairs2['pdb_chain'].nunique()

438

In [7]:
prot_cancer = pd.read_csv('E:/Projek pak Wisnu/Projek Peptide-Protein Interaction/frequently-mutated-genes cancer.csv')
prot_cancer

,Symbol,Name,Annotations,Survival,Uniprot_Accession
0,SEPT5,septin 5,Cancer Gene Census,add to survival plot,Q99719
1,SEPT6,septin 6,Cancer Gene Census,add to survival plot,Q14141
2,SEPT9,septin 9,Cancer Gene Census,add to survival plot,Q9UHD8
3,ABI1,abl-interactor 1,Cancer Gene Census,add to survival plot,Q8IZP0
4,ABL1,"ABL proto-oncogene 1, non-receptor tyrosine ki...",Cancer Gene Census,add to survival plot,P00519
...,...,...,...,...,...
567,ZMYM2,"zinc finger, MYM-type 2",Cancer Gene Census,add to survival plot,Q9UBW7
568,ZNF331,zinc finger protein 331,Cancer Gene Census,add to survival plot,Q9NQX6
569,ZNF384,zinc finger protein 384,Cancer Gene Census,add to survival plot,Q8TF68
570,ZNF521,zinc finger protein 521,Cancer Gene Census,add to survival plot,Q96K83


In [10]:
prot_cancer[~prot_cancer['Uniprot_Accession'].isin(pep_pairs['Uniprot_id'])]

,Symbol,Name,Annotations,Survival,Uniprot_Accession
0,SEPT5,septin 5,Cancer Gene Census,add to survival plot,Q99719
1,SEPT6,septin 6,Cancer Gene Census,add to survival plot,Q14141
2,SEPT9,septin 9,Cancer Gene Census,add to survival plot,Q9UHD8
3,ABI1,abl-interactor 1,Cancer Gene Census,add to survival plot,Q8IZP0
5,ABL2,"ABL proto-oncogene 2, non-receptor tyrosine ki...",Cancer Gene Census,add to survival plot,P42684
...,...,...,...,...,...
567,ZMYM2,"zinc finger, MYM-type 2",Cancer Gene Census,add to survival plot,Q9UBW7
568,ZNF331,zinc finger protein 331,Cancer Gene Census,add to survival plot,Q9NQX6
569,ZNF384,zinc finger protein 384,Cancer Gene Census,add to survival plot,Q8TF68
570,ZNF521,zinc finger protein 521,Cancer Gene Census,add to survival plot,Q96K83


In [ ]:
df_uni2seq = pd.read_csv('uniprot2seq.tsv',sep = '\t',header = 0)

In [ ]:
df_predict_det2_no_uni

## Step 2 Peptide and Protein Feature Extraction (Intrinsic Disorder, PSSM, and SS)

In [77]:
import pandas as pd
import numpy as np
import pickle
# Load Intrinsic disorder
# dict : {sequence: Intrinsic Disorder Matrix}
# Intrinsic Disorder Matrix : (sequence length ,3) , last dimension :(long , short, ANCHOR score)

def extract_intrinsic_disorder(filename,ind):
    fasta_filename = filename+'.fasta'
    disorder_filename = filename+'_'+ind+'.result'
    raw_fasta_list = []
    with open('E:/example_data_feature/Intrinsic Disorder/' + fasta_filename,'r',encoding='utf-8-sig') as f:
        for line in f.readlines():
            line_list = line.strip()
            raw_fasta_list.append(line_list)

    fasta_id_list = [x for x in raw_fasta_list if x[0]=='>']
    fasta_sequence_list = [x for x in raw_fasta_list if x[0]!='>']
    fasta_seq_len_list = [len(x) for x in fasta_sequence_list]
    print(len(fasta_id_list),len(fasta_sequence_list),len(fasta_seq_len_list))

    fasta_dict = {}
    for i in range(len(fasta_id_list)):
        fasta_dict[fasta_id_list[i]]=(fasta_sequence_list[i],fasta_seq_len_list[i])
    # load protein intrinsic disorder result
    raw_result_list = []
    with open('E:/example_data_feature/Intrinsic Disorder/' + disorder_filename,'r',encoding='utf-8-sig') as f:
            for line in f.readlines():
                line_list = line.strip()
                if (len(line_list)>0 and line_list[0]!='#'):
                    raw_result_list.append(line_list)

    intrinsic_id_list = [x for x in raw_result_list if x[0]=='>']
    intrinsic_score_list = [x.split('\t') for x in raw_result_list if x[0]!='>']
    start_idx = 0
    raw_score_dict = {}
    for idx in range(len(intrinsic_id_list)):
        prot_id = intrinsic_id_list[idx]
        seq_len = fasta_dict[prot_id][1]
        end_idx = start_idx + seq_len
        individual_score_list = intrinsic_score_list[start_idx:end_idx]
        individual_score_list=[x[2:] for x in individual_score_list]
        individual_score_array = np.array(individual_score_list,dtype='float')
        raw_score_dict[prot_id] = individual_score_array
        start_idx = end_idx
    print(len(fasta_dict.keys()),len(raw_score_dict.keys()))
    return fasta_dict, raw_score_dict

# long & short
fasta_dict_long, raw_score_dict_long = extract_intrinsic_disorder('input4','long') # the input fasta file used in IUPred2A
fasta_dict_short, raw_score_dict_short = extract_intrinsic_disorder('input4','short')


Intrinsic_score_long = {}
for key in fasta_dict_long.keys():
    sequence = fasta_dict_long[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_long[key][1]
    Intrinsic = raw_score_dict_long[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_long[id_fasta] = Intrinsic

    
Intrinsic_score_short = {}
for key in fasta_dict_short.keys():
    sequence = fasta_dict_short[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_short[key][1]
    Intrinsic = raw_score_dict_short[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_short[id_fasta] = Intrinsic
   

Intrinsic_score = {}
for seq in Intrinsic_score_short.keys():
    long_Intrinsic = Intrinsic_score_long[seq]
    short_Intrinsic = Intrinsic_score_short[seq]
    concat_Intrinsic = np.column_stack((long_Intrinsic,short_Intrinsic))
    Intrinsic_score[seq] = np.column_stack((long_Intrinsic,short_Intrinsic))

# print(Intrinsic_score)
with open('E:/example_data_feature/Intrinsic Disorder/output_intrisic_dict4','wb') as f: # 'output_intrisic_dict' is the name of the output dict you like
    pickle.dump(Intrinsic_score,f)

8498 8498 8498
6989 6989
8498 8498 8498
6989 6989


In [43]:
#PROT CANCER

import pandas as pd
import numpy as np
import pickle
# Load Intrinsic disorder
# dict : {sequence: Intrinsic Disorder Matrix}
# Intrinsic Disorder Matrix : (sequence length ,3) , last dimension :(long , short, ANCHOR score)

def extract_intrinsic_disorder(filename,ind):
    fasta_filename = filename+'.fasta'
    disorder_filename = filename+'_'+ind+'.result'
    raw_fasta_list = []
    with open('E:/temp/' + fasta_filename,'r',encoding='utf-8-sig') as f:
        for line in f.readlines():
            line_list = line.strip()
            raw_fasta_list.append(line_list)

    fasta_id_list = [x for x in raw_fasta_list if x[0]=='>']
    fasta_sequence_list = [x for x in raw_fasta_list if x[0]!='>']
    fasta_seq_len_list = [len(x) for x in fasta_sequence_list]
    print(len(fasta_id_list),len(fasta_sequence_list),len(fasta_seq_len_list))

    fasta_dict = {}
    for i in range(len(fasta_id_list)):
        fasta_dict[fasta_id_list[i]]=(fasta_sequence_list[i],fasta_seq_len_list[i])
    # load protein intrinsic disorder result
    raw_result_list = []
    with open('E:/temp/' + disorder_filename,'r',encoding='utf-8-sig') as f:
            for line in f.readlines():
                line_list = line.strip()
                if (len(line_list)>0 and line_list[0]!='#'):
                    raw_result_list.append(line_list)

    intrinsic_id_list = [x for x in raw_result_list if x[0]=='>']
    intrinsic_score_list = [x.split('\t') for x in raw_result_list if x[0]!='>']
    start_idx = 0
    raw_score_dict = {}
    for idx in range(len(intrinsic_id_list)):
        prot_id = intrinsic_id_list[idx]
        seq_len = fasta_dict[prot_id][1]
        end_idx = start_idx + seq_len
        individual_score_list = intrinsic_score_list[start_idx:end_idx]
        individual_score_list=[x[2:] for x in individual_score_list]
        individual_score_array = np.array(individual_score_list,dtype='float')
        raw_score_dict[prot_id] = individual_score_array
        start_idx = end_idx
    print(len(fasta_dict.keys()),len(raw_score_dict.keys()))
    return fasta_dict, raw_score_dict

# long & short
fasta_dict_long, raw_score_dict_long = extract_intrinsic_disorder('input_prot_cancer2','long') # the input fasta file used in IUPred2A
fasta_dict_short, raw_score_dict_short = extract_intrinsic_disorder('input_prot_cancer2','short')


Intrinsic_score_long = {}
for key in fasta_dict_long.keys():
    sequence = fasta_dict_long[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_long[key][1]
    Intrinsic = raw_score_dict_long[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_long[id_fasta] = Intrinsic

    
Intrinsic_score_short = {}
for key in fasta_dict_short.keys():
    sequence = fasta_dict_short[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_short[key][1]
    Intrinsic = raw_score_dict_short[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_short[id_fasta] = Intrinsic
   

Intrinsic_score = {}
for seq in Intrinsic_score_short.keys():
    long_Intrinsic = Intrinsic_score_long[seq]
    short_Intrinsic = Intrinsic_score_short[seq]
    concat_Intrinsic = np.column_stack((long_Intrinsic,short_Intrinsic))
    Intrinsic_score[seq] = np.column_stack((long_Intrinsic,short_Intrinsic))

# print(Intrinsic_score)
with open('E:/temp/output_intrisic_dict_cancer','wb') as f: # 'output_intrisic_dict' is the name of the output dict you like
    pickle.dump(Intrinsic_score,f)

133 133 133
133 133
133 133 133
133 133


In [98]:
len(Intrinsic_score)

6618

In [44]:
(pd.DataFrame.from_dict(data=Intrinsic_score, orient='index')
   .to_csv('E:/temp/output_intrisic_dict_cancer2.csv', header=False))

D:\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [13]:
IS_df = pd.read_csv('E:/example_data_feature/Intrinsic Disorder/output_intrisic_dict3.csv')
IS_df

,pdb_chain,score
0,1acy_p,[[0.4051 0.3983 0.8823 0.3983]\n [0.4051 0.412...
1,1qgc_5,[[0.4766 0.4187 0.9103 0.4187]\n [0.4087 0.421...
2,1vrk_b,[[0.712 0.6661 0.9562 0.6661]\n [0.7415 0.675...
3,1xcq_p,[[0.9879 0.6143 0.9994 0.6143]\n [0.9875 0.619...
4,2b1a_p,[[0.6269 0.4198 0.928 0.4198]\n [0.6227 0.420...
...,...,...
6980,7wvw_l,[[0. 0. 0.1698 0. ]\n [0. 0. ...
6981,7yxw_d,[[0.9844 0.4129 0.9932 0.4129]\n [0.9797 0.030...
6982,7zez_d,[[0.9269 0.8684 0.9922 0.8684]\n [0.9249 0.842...
6983,8gch_c,[[1. 1. 1. 1.]\n [1. 1. 1. 1.]\n [1. 1. 1. 1.]]


In [103]:
# IS_df['Seq'].nunique()
fasta_sequence_list_df = pd.DataFrame(fasta_sequence_list)
fasta_sequence_list_df[0].nunique()

6036

In [8]:
fasta_filename = 'input2'+'.fasta'
# disorder_filename = filename+'_'+ind+'.result'
raw_fasta_list = []
with open('E:/example_data_feature/Intrinsic Disorder/' + fasta_filename,'r') as f:
    for line in f.readlines():
        line_list = line.strip()
        raw_fasta_list.append(line_list)


fasta_id_list = [x for x in raw_fasta_list if x[0]=='>']
fasta_sequence_list = [x for x in raw_fasta_list if x[0]!='>']


# newline = '\n'
# for i in range(len(fasta_id_list)):
#     fasta_id = fasta_id_list[i].split('>')[1]+'.fasta'
#     stri = f"{fasta_id_list[i]}{newline}{fasta_sequence_list[i]}"
#     with open('E:/example_data_feature/PSSM/input/'+fasta_id,'w') as f:
#         f.writelines(stri)

#     with open('E:/output_intrisic_dict','wb') as f:
#         fasta_seq_len_list = [len(x) for x in fasta_sequence_list]
# print(len(fasta_id_list),len(fasta_sequence_list),len(fasta_seq_len_list))

In [ ]:
for i in range(7606,8497):
    fasta_id = fasta_id_list[i].split('>')[1]
    print(fasta_id)

In [ ]:
import subprocess
import os
for i in range(7607,8497):
    fasta_id = fasta_id_list[i].split('>')[1]
    print(fasta_id)
    os.popen(f'psiblast -query E:/example_data_feature/PSSM/input/{fasta_id}.fasta -db nr -out_ascii_pssm E:/example_data_feature/PSSM/kurang/{fasta_id}.pssm -remote').read()

6ji2


## Multiprocess

In [5]:
import pandas as pd
import numpy as np
pssmkurang = pd.read_csv('pssmkurang.csv')

In [6]:
from os import walk
_, _, filenames = next(walk('E:/example_data_feature/PSSM/output_protein3'), (None, None, []))

In [7]:
list_file=[x.split('.')[0] for x in filenames]
list_file

[]

In [8]:
pep_pairs2[~pep_pairs2['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].nunique()

133

In [ ]:
pep_pairs2

In [9]:
import multiprocess as mp
import os

def pssm_extract(Uniprot_chain):
    import os
    print(Uniprot_chain)
    os.popen(f'psiblast -query E:/example_data_feature/PSSM/input_protein/{Uniprot_chain}.fasta -db refseq_protein -num_iterations 3 -out_ascii_pssm E:/example_data_feature/PSSM/output_protein3/{Uniprot_chain}.pssm -entrez_query "Homo Sapiens [ORGN]" -remote').read()

with mp.Pool(os.cpu_count() - 2) as pool:
    pool.map(pssm_extract, pep_pairs2[~pep_pairs2['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].unique())
    pool.close()

In [35]:
pep_pairs['pdb_id'].nunique()

6618

In [13]:
pep_pairs2['Uniprot_chain'].nunique()

133

In [7]:
import numpy as np
### Load Protein PSSM Files (first change the value of protein_number)
# prot_pssm_dict : key is protein sequence, value is protein PSSM Matrix
prot_pssm_dict_all={}
pep_not_found = []
prot_pssm_dict={}
prot_pssm_arr = []
pep_list = pep_pairs2['Uniprot_chain'].unique()
protein_num = 0 ### NEED TO BE CHANGED TO the total number of protein sequences
for i in range(len(pep_list)):
    try:
        filename_pssm = pep_list[i]+'.pssm' # need to name each individual fasta and pssm file with the same prefix
        filename_fasta = pep_list[i]+'.fasta'
        prot_key = pep_list[i]
        pssm_line_list= []
        print(filename_pssm)
        with open('E:/example_data_feature/PSSM/input_protein/'+str(filename_fasta),'r') as f: # directory to store fasta files (single file of each protein)
            for line in f.readlines():
                prot_seq = line.strip()

        with open('E:/example_data_feature/PSSM/output_protein/'+str(filename_pssm),'r') as f:  # directory to store pssm files (single file of each protein)
            for line in f.readlines()[3:-4]:
                line_list = line.strip().split(' ')
                line_list = [x for x in line_list if x!=''][2:22]
                if (len(line_list)!=20 or line_list[0]=='-I'):
#                     print('Error line:')
#                     print(line_list)
                    continue
                line_list = [int(x) for x in line_list]
                pssm_line_list.append(line_list)
            pssm_array = np.array(pssm_line_list)
            if pssm_array.shape[1]!=20:
                print('Error!')
                print(filename_pssm)
            else:
                print(pssm_array)
                prot_pssm_dict_all[prot_key] = (prot_seq,pssm_array)
                prot_pssm_dict[prot_seq]=pssm_array
                prot_pssm_arr.append(pssm_array)
    except:
        print('Pep not found with ID:', pep_list[i])
        pep_not_found.append(pep_list[i])


Q15796_A.pssm
[[ 4 -1 -2 ... -3 -2  0]
 [-2 -1  4 ... -4 -3 -3]
 [ 0 -3 -3 ... -2 -2 -1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
P37231_D.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [ -3  -2  -3 ...   2 554  -1]
 [ -2  -2  11 ...  -2   0  -1]
 [ -1  -1  -1 ...  -1   0  -1]]
P04233_I.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  0  -1  -1 ...  -4   3  -1]
 [ -2  -2  -2 ...  -1 609  -4]
 [  6  -3  -3 ...  -1 609  -4]]
P35222_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  0   0   0 ...   0   0   0]
 [  0 644   0 ...   0 644 644]
 [  0   0   0 ... 644   0   0]]
Q01081_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [ -3  -3  -2 ...   3 526  -1]
 [ -3  -4  -1 ... 526   0  -3]
 [ -4  -3  -1 ... 526   0  -4]]
Q92793

[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  6  -3  -3 ...  -1 564  -4]
 [ -1  -1  -1 ...  -1 564  -1]
 [  0   4  -2 ...  -3 564  -1]]
Q15788_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [335 299  39 ...   0   0   0]
 [ 85  23 343 ...   0   0   0]
 [420 420 420 ...   0   0   0]]
P42345_C.pssm
[[ 4 -1 -2 ... -3 -2  0]
 [-2 -1  4 ... -4 -3 -3]
 [ 0 -3 -3 ... -2 -2 -1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
P01116_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  0   0   0 ...   0   0 310]
 [  0   0   0 ...   0   0 310]
 [  0   0   0 ...   0   0   0]]
P01112_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  0  -1  -1 ...  -4   3  -1]
 [ -2  -2  -2 ...  -1 636  -4]
 [  6  -3  -3 ...  -1 636  -4]]
Q8ND56_A.pssm
[[ 4 -

[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [  0  -1  -1 ...  -4   3  -1]
 [ -2  -2  -2 ...  -1 370  -4]
 [  6  -3  -3 ...  -1 370  -4]]
Q00987_B.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [ -2   0 593 ...   0   0   7]
 [ -3  15   0 ...  40   0   0]
 [  0   0 593 ...   0   0 593]]
Q9UMX1_A.pssm
[[ 4 -1 -2 ... -3 -2  0]
 [-2 -1  4 ... -4 -3 -3]
 [ 0 -3 -3 ... -2 -2 -1]
 ...
 [ 1  0 -4 ...  0  0  0]
 [ 0  0 -4 ...  0  0  0]
 [-3  0 -4 ...  0  0  0]]
Q9BZ95_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [ -3  -3  -2 ...   3 584  -1]
 [ -3  -4  -1 ... 584   0  -3]
 [ -4  -3  -1 ... 584   0  -4]]
O43791_A.pssm
[[  4  -1  -2 ...  -3  -2   0]
 [ -2  -1   4 ...  -4  -3  -3]
 [  0  -3  -3 ...  -2  -2  -1]
 ...
 [ -1   0   0 ...   0   0   1]
 [ -4   0 408 ...   0   0 408]
 [ -4  28   0 ...   5   1   0]]
Q9Y6K1_A.pssm
[[  4 

In [8]:
prot_pssm_dict

{'MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVKSLVKKLKKTGRLDELEKAITTQNCNTKCVTIPSTCSEIWGLSTPNTIDQWDTTGLYSFSEQTRSLDGRLQVSHRKGLPHVIYCRLWRWPDLHSHHELKAIENCEYAFNLKKDEVCVNPYHYQRVETPVLPPVLVPRHTEILTELPPLDDYTHSIPENTNFPAGIEPQSNYIPETPPPGYISEDGETSDQQLNQSMDTGSPAELSPTTLSPVNHSLDLQPVTYSEPAFWCSIAYYELNQRVGETFHASQPSLTVDGFTDPSNSERFCLGLLSNVNRNATVEMTRRHIGRGVRLYYIGGEVFAECLSDSAIFVQSPNCNQRYGWHPATVCKIPPGCNLKIFNNQEFAALLAQSVNQGFEAVYQLTRMCTIRMSFVKGWGAEYRRQTVTSTPCWIELHLNGPLQWLDKVLTQMGSPSVRCSSMSMSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVKSLVKKLKKTGRLDELEKAITTQNCNTKCVTIPSTCSEIWGLSTPNTIDQWDTTGLYSFSEQTRSLDGRLQVSHRKGLPHVIYCRLWRWPDLHSHHELKAIENCEYAFNLKKDEVCVNPYHYQRVETPVLPPVLVPRHTEILTELPPLDDYTHSIPENTNFPAGIEPQSNYIPETPPPGYISEDGETSDQQLNQSMDTGSPAELSPTTLSPVNHSLDLQPVTYSEPAFWCSIAYYELNQRVGETFHASQPSLTVDGFTDPSNSERFCLGLLSNVNRNATVEMTRRHIGRGVRLYYIGGEVFAECLSDSAIFVQSPNCNQRYGWHPATVCKIPPGCNLKIFNNQEFAALLAQSVNQGFEAVYQLTRMCTIRMSFVKGWGAEYRRQTVTSTPCWIELHLNGPLQWLDKVLTQMGSPSVRCSSMSMSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVKSLVKKLKKTGRLDELEKA

In [ ]:
#PSSM PROT CCANCER

import numpy as np
### Load Protein PSSM Files (first change the value of protein_number)
# prot_pssm_dict : key is protein sequence, value is protein PSSM Matrix
prot_pssm_dict_all={}
pep_not_found = []
prot_pssm_dict={}
prot_pssm_arr = []
pep_list = pssmkurang['pdb_chain']
protein_num = 0 ### NEED TO BE CHANGED TO the total number of protein sequences
for i in range(len(pep_list)):
    try:
        filename_pssm = pep_list[i]+'.pssm' # need to name each individual fasta and pssm file with the same prefix
        filename_fasta = pep_list[i]+'.fasta'
        prot_key = pep_list[i]
        pssm_line_list= []
        print(filename_pssm)
        with open('E:/example_data_feature/PSSM/input2/'+str(filename_fasta),'r') as f: # directory to store fasta files (single file of each protein)
            for line in f.readlines():
                prot_seq = line.strip()

        with open('E:/example_data_feature/PSSM/kurang/'+str(filename_pssm),'r') as f:  # directory to store pssm files (single file of each protein)
            for line in f.readlines()[3:-4]:
                line_list = line.strip().split(' ')
                line_list = [x for x in line_list if x!=''][2:22]
                if (len(line_list)!=20 or line_list[0]=='-I'):
#                     print('Error line:')
#                     print(line_list)
                    continue
                line_list = [int(x) for x in line_list]
                pssm_line_list.append(line_list)
            pssm_array = np.array(pssm_line_list)
            if pssm_array.shape[1]!=20:
                print('Error!')
                print(filename_pssm)
            else:
                print(pssm_array)
                prot_pssm_dict_all[prot_key] = (prot_seq,pssm_array)
                prot_pssm_dict[prot_seq]=pssm_array
                prot_pssm_arr.append(pssm_array)
    except:
        print('Pep not found with ID:', pep_list[i])
        pep_not_found.append(pep_list[i])


In [30]:
pep_pairs2[~pep_pairs2['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].unique()

array(['Q15796_A', 'P35222_A', 'Q92793_B', 'Q09472_B', 'P00533_A',
       'P10275_A', 'P30622_A', 'P37231_A', 'Q14203_A', 'P42566_A',
       'P42768_A', 'Q09472_A', 'Q92793_A', 'Q86U86_B', 'P42568_A',
       'O15550_A', 'P25054_A', 'O15164_A', 'Q9UPN9_A', 'Q06455_A',
       'Q92794_A', 'Q9Y4I1_A', 'Q8NG31_A', 'Q7KZF4_A', 'P35240_A',
       'Q8NEZ4_C', 'Q03164_A', 'Q9UER7_A', 'P22681_A', 'Q03111_A',
       'Q15788_A', 'P42345_C', 'Q8NEZ4_A', 'Q15059_A', 'O15151_A',
       'O75533_C', 'P43405_A', 'P00519_A', 'P06239_A', 'Q12923_A',
       'P03372_A', 'P27986_A', 'P06400_A', 'P41182_A', 'P38398_A',
       'Q00987_A', 'P08575_A', 'P55196_A', 'P29375_A', 'Q14839_A',
       'P46100_A', 'Q9HD26_A', 'O60885_B', 'O60885_A', 'P52948_A',
       'O15164_B', 'P22681_B', 'Q9ULV8_A', 'Q13191_A', 'Q06124_A',
       'Q9BZ95_A', 'O43791_A', 'Q9Y6K1_A', 'P55197_B', 'Q15910_A',
       'Q9BYW2_A', 'P08581_A', 'Q8WYB5_B', 'P03372_B', 'Q15059_B',
       'O75376_A', 'Q00610_A', 'Q92793_J', 'Q03164_B'], dtype=

In [6]:
import subprocess
import os
for Uniprot_chain in pep_pairs2[~pep_pairs2['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].unique():
    print(Uniprot_chain)
    os.popen(f'psiblast -query E:/example_data_feature/PSSM/input_protein/{Uniprot_chain}.fasta -db refseq_protein -out_ascii_pssm E:/example_data_feature/PSSM/output_protein/{Uniprot_chain}.pssm -entrez_query "Homo Sapiens [ORGN]" -remote').read()

P35222_A
Q92793_B
Q09472_B
P00533_A
P10275_A
P30622_A
P37231_A
Q14203_A
P42566_A
P42768_A
Q09472_A
Q92793_A
Q86U86_B
P42568_A
O15550_A
P25054_A
O15164_A
Q9UPN9_A
Q06455_A
Q92794_A
Q9Y4I1_A
Q8NG31_A
Q7KZF4_A
P35240_A
Q8NEZ4_C
Q03164_A
Q9UER7_A
P22681_A
Q03111_A
Q15788_A
P42345_C
Q8NEZ4_A
Q15059_A
O15151_A
O75533_C
P43405_A
P00519_A
P06239_A
Q12923_A
P03372_A
P27986_A
P06400_A
P41182_A
P38398_A
Q00987_A
P08575_A
P55196_A
P29375_A
Q14839_A
P46100_A
Q9HD26_A
O60885_B
O60885_A
P52948_A
O15164_B
P22681_B
Q9ULV8_A
Q13191_A
Q06124_A
Q9BZ95_A
O43791_A
Q9Y6K1_A
P55197_B
Q15910_A
Q9BYW2_A
P08581_A
Q8WYB5_B
P03372_B
Q15059_B
O75376_A
Q00610_A
Q92793_J
Q03164_B


In [68]:
import pickle
with open('E:/example_data_feature/PSSM/output_pssm_dict_cancer','wb') as f:  # 'output_pssm_dict' is the name of the output dict you like
    pickle.dump(prot_pssm_dict,f)

import pandas as pd
(pd.DataFrame.from_dict(data=prot_pssm_dict_all, orient='index')
   .to_csv('E:/example_data_feature/PSSM/output_pssm_dict_cancer.csv', header=False))

In [43]:
pssm_df2= pd.DataFrame.from_dict(data=prot_pssm_dict_all, orient='index', columns = ['Seq','matrix'])
pssm_df2

,Seq,matrix
1acy,FHCVPRDLSWLDLEANMCLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1qgc,GAEDAQDDLVPSIQDDGSESGACKI,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1vrk,YNKRKRIHIGPGRAFYTTKNIIGC,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1xcq,RRKWQKTGHAVRAIGRLSSSX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2b1a,STNPKPQRKTKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLG,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
7wvw,MLFII,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7yxw,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7zez,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
8gch,PPSNPPPRPPAEARKKPS,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [20]:
pssm_df= pd.DataFrame.from_dict(data=prot_pssm_dict_all, orient='index', columns = ['Seq','matrix'])
pssm_df

,Seq,matrix
Q15796_A,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
P37231_D,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
P04233_I,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
P35222_A,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
Q01081_A,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
O43791_B,MSRVPSPPPPAEMSSGPVAESWCYTQIKVVKFSYMWTINNFSFCRE...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
P04637_B,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
P19174_A,MAGAASPCANGCGPGAPSDAEVLHLCRSLEVGTVMTLFYSKKSQRP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
Q92793_J,MAENLLDGPPNPKRAKLSSPGFSANDSTDFGSLFDLENDLPDELIP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [21]:
pssm_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
pssm_df

,Seq,matrix,Uniprot_chain
Q15796_A,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q15796_A
P37231_D,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P37231_D
P04233_I,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04233_I
P35222_A,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P35222_A
Q01081_A,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q01081_A
...,...,...,...
O43791_B,MSRVPSPPPPAEMSSGPVAESWCYTQIKVVKFSYMWTINNFSFCRE...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",O43791_B
P04637_B,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04637_B
P19174_A,MAGAASPCANGCGPGAPSDAEVLHLCRSLEVGTVMTLFYSKKSQRP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P19174_A
Q92793_J,MAENLLDGPPNPKRAKLSSPGFSANDSTDFGSLFDLENDLPDELIP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q92793_J


In [52]:
pssm_gabung = pd.concat([pssm_df,pssm_df2],join='inner')
pssm_gabung['Seq'].nunique()

6036

In [53]:
pssm_gabung_duplicate = pssm_gabung[pssm_gabung.duplicated(['Seq'])]
print("Duplicate Rows based on 2 columns are:", pssm_gabung_duplicate, sep='\n')
pssm_gabung_clean = pssm_gabung.drop_duplicates(subset=['Seq'])
# drugcentral_clean = supertarget_clean.rename(columns={'Drug': 'Drug Name'})
pssm_gabung_clean

Duplicate Rows based on 2 columns are:
                                                      Seq  \
2pye_c                                          SLLMWITQC   
2axt_t                   METITYVFIFACIIALFFFAIFFREPPRITKK   
3j5s_h  GIREKIKLVSSAGTGHFYTTTKNKRTKPEKLELKKFDPVVRQHVIY...   
3wu2_t                   METITYVFIFACIIALFFFAIFFREPPRITKK   
4fby_t                   METITYVFIFACIIALFFFAIFFREPPRITKK   
...                                                   ...   
7vfl                                          RPKPQQFFGLM   
7wvv                                      LTERHKILHRLLQEG   
7wvw                                                MLFII   
7zez                                            MYFINILTL   
8hvp                                        ARTKQTARKSTGG   

                                                   matrix  
2pye_c  [[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...  
2axt_t  [[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...  
3j5s_h  [[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -

,Seq,matrix
2kj4_b,GSVEKLTADAELQRLKNERHEEAELERLKSEY,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2l53_b,GPGSEEVSAMVIQRAFRRHLLQRSLKHASFL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3he5_a,GSNLVAQLENEVASLENENETLKKKNLHKKDLIAYLEKEIANLRKKIEE,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3o40_b,XXTWEXWDXAIAEYAARIEALIRAAQEQQEKNEXALXELX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
4dzu_b,RTWEEWDRAIAEYARRIEELIRAAQEQQRKNEEALREL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
7w2z,QTARKS,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7wgo,TRRETQL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7wui,GSXFLSPEHQRVQQRK,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7yxw,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [58]:
pssm_gabung_clean = pssm_gabung_clean.rename({'Seq':'pep_seq'},axis=1)

In [59]:
pssm_gabung_clean

,pep_seq,matrix
2kj4_b,GSVEKLTADAELQRLKNERHEEAELERLKSEY,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2l53_b,GPGSEEVSAMVIQRAFRRHLLQRSLKHASFL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3he5_a,GSNLVAQLENEVASLENENETLKKKNLHKKDLIAYLEKEIANLRKKIEE,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3o40_b,XXTWEXWDXAIAEYAARIEALIRAAQEQQEKNEXALXELX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
4dzu_b,RTWEEWDRAIAEYARRIEELIRAAQEQQRKNEEALREL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
7w2z,QTARKS,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7wgo,TRRETQL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7wui,GSXFLSPEHQRVQQRK,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7yxw,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [38]:
pssm_df2 = pd.DataFrame()
pssm_df2['peptide']=pep_pairs['pdb_id'].unique()
pssm_df2['pssm']=prot_pssm_arr

In [39]:
pssm_df2

,peptide,pssm
0,1acy,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1,1qgc,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2,1vrk,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3,1xcq,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
4,2b1a,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
6613,7wvw,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6614,7yxw,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6615,7zez,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6616,8gch,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [27]:
pssm_df = pd.DataFrame()
pssm_df['peptide']=pssmkurang['pdb_chain'].unique()
pssm_df['pssm']=prot_pssm_arr

In [55]:
pssm_pssm = pd.read_csv('pssmfix.csv')
pssm_pssm = pssm_pssm.drop(['matrix'],axis=1)
pssm_pssm

,pdb_chain,pep_seq
0,1acy_p,YNKRKRIHIGPGRAFYTTKNIIGC
1,1ai1_p,YNKRKRIHIGPGRAFYTTKNIIGC
2,1qgc_5,TTAYTASARGDLAHLTTTAARTLP
3,1vrk_b,RRKWQKTGHAVRAIGRLSSSX
4,1qs7_b,RRKWQKTGHAVRAIGRLSSSX
...,...,...
6984,7t6u_l,QKFTSWFX
6985,7tbh_a,LSRRPSYRKILNDLSSDAP
6986,7wui_l,ISFGILLDLSRTSLP
6987,8gch_c,GAW


In [60]:
pssm_complete = pd.merge(pssm_pssm, pssm_gabung_clean,  on="pep_seq", how='left')
pssm_complete

,pdb_chain,pep_seq,matrix
0,1acy_p,YNKRKRIHIGPGRAFYTTKNIIGC,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1,1ai1_p,YNKRKRIHIGPGRAFYTTKNIIGC,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2,1qgc_5,TTAYTASARGDLAHLTTTAARTLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3,1vrk_b,RRKWQKTGHAVRAIGRLSSSX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
4,1qs7_b,RRKWQKTGHAVRAIGRLSSSX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...,...
6984,7t6u_l,QKFTSWFX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6985,7tbh_a,LSRRPSYRKILNDLSSDAP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6986,7wui_l,ISFGILLDLSRTSLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
6987,8gch_c,GAW,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [29]:
pssm_df

,Seq,matrix,Uniprot_chain
Q15796_A,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q15796_A
P37231_D,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P37231_D
P04233_I,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04233_I
P35222_A,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P35222_A
Q01081_A,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q01081_A
...,...,...,...
O43791_B,MSRVPSPPPPAEMSSGPVAESWCYTQIKVVKFSYMWTINNFSFCRE...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",O43791_B
P04637_B,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04637_B
P19174_A,MAGAASPCANGCGPGAPSDAEVLHLCRSLEVGTVMTLFYSKKSQRP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P19174_A
Q92793_J,MAENLLDGPPNPKRAKLSSPGFSANDSTDFGSLFDLENDLPDELIP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q92793_J


# Auto Covariance PSSM

In [22]:
def row_to_col(X):
    transform_mat = []
    for i in range(20):
        col = []
        for j in range(len(X)):
            col.append(X[j][i])
        transform_mat.append(col)
    transform_mat = np.array(transform_mat)
    return transform_mat

In [23]:
def lagged_auto_cov(Xi,t):
    """
    for series of values x_i, length N, compute empirical auto-cov with lag t
    defined: 1/(N-1) * \sum_{i=0}^{N-t} ( x_i - x_s ) * ( x_{i+t} - x_s )
    """
    N = len(Xi)
    # use sample mean estimate from whole series
    Xs = np.mean(Xi)

    # construct copies of series shifted relative to each other, 
    # with mean subtracted from values
    end_padded_series = np.zeros(N+t)
    end_padded_series[:N] = Xi - Xs
    start_padded_series = np.zeros(N+t)
    start_padded_series[t:] = Xi - Xs

    auto_cov = 1./(N-1) * np.sum( start_padded_series*end_padded_series )
    return auto_cov


In [66]:
pssm_complete['matrix'][0].shape

(23, 20)

In [24]:
pssm_ac_all = []
for k in range(len(pssm_df['matrix'])):
    pssm_ac_row = []
    print(pssm_df['Uniprot_chain'][k])
    if(pssm_df['matrix'][k].shape[0]==1):
        pssm_ac_all.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    else:
        trans_mat = row_to_col(pssm_df['matrix'][k])
        for l in range(trans_mat.shape[0]):
            ac = lagged_auto_cov(trans_mat[l],1)
            pssm_ac_row.append(ac)
        pssm_ac_all.append(pssm_ac_row)

Q15796_A
P37231_D
P04233_I
P35222_A
Q01081_A
Q92793_B
Q09472_B
P40337_V
P40337_C
P84022_A
P00533_A
P26038_A
P10275_A
P30622_A
P37231_A
Q14203_A
P42566_A
P42768_A
Q09472_A
Q92793_A
Q86U86_B
O14980_B
O75475_B
P42568_A
P42568_B
P26196_A
P10415_A
P25791_A
O15550_A
P25054_A
O15164_A
O60566_A
Q9UPN9_A
O00255_A
Q96E14_B
Q06455_A
Q92794_A
P27540_B
Q9Y4I1_A
Q15276_B
Q8NG31_A
P27540_A
Q7KZF4_A
P25685_A
P35240_A
P26196_B
Q8NEZ4_C
Q03164_A
Q9UER7_A
P22681_A
Q03111_A
P23458_A
Q15788_A
P42345_C
P01116_A
P01112_A
Q8ND56_A
P30153_A
P15923_A
Q8NEZ4_A
P37231_B
P10276_B
P10276_A
P04637_A
Q15059_A
O15151_A
O75533_C
P43405_A
P00519_A
P06239_A
P06239_L
Q12923_A
O15169_A
Q14790_A
P06400_B
P03372_A
O96017_A
P27986_A
P06400_A
P31751_A
P41182_A
P38398_A
Q00987_M
P37231_X
Q00987_A
P08575_A
P55196_A
P11309_B
P62258_A
P29375_A
Q14839_A
P46100_A
Q9HD26_A
O60885_B
O60885_A
P21802_A
P52948_A
P11309_A
P31749_A
O15164_B
P22681_B
P61073_A
Q9ULV8_A
Q13191_A
Q06124_A
Q00987_B
Q9UMX1_A
Q9BZ95_A
O43791_A
Q9Y6K1_A
P55197_B
P

In [28]:
pssm_ac_all = np.array(pssm_ac_all)
pssm_ac_all_df = pd.DataFrame([x for x in pssm_ac_all])
pssm_ac_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
pssm_ac_all_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Uniprot_chain
0,3.512009e+15,0.364231,0.260286,0.216817,711.721899,0.318551,0.230239,0.283537,0.146958,0.245551,...,0.229679,0.229079,0.299297,0.322775,0.299314,0.285005,712.786436,1413.018827,711.673340,Q15796_A
1,-8.437078e-01,0.078755,0.022075,-0.371356,-0.749673,0.176505,-0.328890,-0.104210,-0.392944,-0.226434,...,-0.154844,-0.070623,-0.366098,0.138201,0.093504,0.322820,0.432394,10390.088842,-0.518398,P37231_D
2,-9.422023e-01,0.193447,0.070978,-0.421639,-0.719205,0.182807,-0.405052,-0.130810,-0.359298,-0.239797,...,-0.107196,-0.001130,-0.343482,0.144338,0.085292,0.024647,0.394863,2276.239173,-0.538727,P04233_I
3,-1.573697e+13,8252.145994,6373.101040,1100.099798,2222.212605,7086.538630,1084.107364,1084.982288,1102.419496,1083.312224,...,2150.810827,6422.584629,1100.352180,10161.851417,9918.636416,9818.299951,14354.998022,17386.117793,14084.097444,P35222_A
4,-8.826374e-01,0.300049,0.140303,-0.535672,-0.609397,0.149232,-0.484681,-0.139879,-0.325122,-0.178137,...,-0.069944,-0.196855,-0.272132,0.145116,0.123932,-0.021888,1651.850706,6399.426367,-0.510186,Q01081_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,-9.422023e-01,0.193447,0.070978,-0.421639,-0.719205,0.182807,-0.405052,-0.130810,-0.359298,-0.239797,...,-0.107196,-0.001130,-0.343482,0.144338,0.085292,0.024647,0.394863,2087.229403,-0.538727,O43791_B
129,-9.422023e-01,0.193447,0.070978,-0.421639,-0.719205,0.182807,-0.405052,-0.130810,-0.359298,-0.239797,...,-0.107196,-0.001130,-0.343482,0.144338,0.085292,0.024647,0.394863,1167.365189,-0.538727,P04637_B
130,-9.422023e-01,0.193447,0.070978,-0.421639,-0.719205,0.182807,-0.405052,-0.130810,-0.359298,-0.239797,...,-0.107196,-0.001130,-0.343482,0.144338,0.085292,0.024647,0.394863,1913.214770,-0.538727,P19174_A
131,5.157205e+05,8680.886322,9333.578267,9912.289541,9249.669753,7135.370476,10257.158529,9980.628486,10969.731085,8922.317424,...,9233.956728,8713.641042,8646.491788,9552.482727,9213.882102,7594.738009,9095.904438,5716.820973,7374.115614,Q92793_J


In [30]:
pssm_ac_all_df.to_csv('pssm_complete_ac_cancer.csv',index=False)

# Cross Covariance PSSM

In [31]:
def crosscovariance(Xi,Yi,k):
    N = np.size(Xi)
    Xs = np.average(Xi)
    Ys = np.average(Yi)
    crossCov = 0
    for i in np.arange(0, N-k):
        crossCov += ((Yi[i+k])-Xs)*(Xi[i]-Xs)
    return (1/(N-1))*crossCov

In [32]:
pssm_cc_all = []
for k in range(len(pssm_df['matrix'])):
    print(pssm_df['Uniprot_chain'][k])
    if(pssm_df['matrix'][k].shape[0]==1):
        pssm_cc_all.append([0]*380)
    else:
        trans_mat = row_to_col(pssm_df['matrix'][k])
        pssm_cc_row = []
        for l in range(trans_mat.shape[0]):
            for m in range(trans_mat.shape[0]):
                if (l==m):
                    continue
                else:
                    cc = crosscovariance(trans_mat[l],trans_mat[m],1)
                    pssm_cc_row.append(cc)
        pssm_cc_all.append(pssm_cc_row)

Q15796_A
P37231_D
P04233_I
P35222_A
Q01081_A
Q92793_B
Q09472_B
P40337_V
P40337_C
P84022_A
P00533_A
P26038_A
P10275_A
P30622_A
P37231_A
Q14203_A
P42566_A
P42768_A
Q09472_A
Q92793_A
Q86U86_B
O14980_B
O75475_B
P42568_A
P42568_B
P26196_A
P10415_A
P25791_A
O15550_A
P25054_A
O15164_A
O60566_A
Q9UPN9_A
O00255_A
Q96E14_B
Q06455_A
Q92794_A
P27540_B
Q9Y4I1_A
Q15276_B
Q8NG31_A
P27540_A
Q7KZF4_A
P25685_A
P35240_A
P26196_B
Q8NEZ4_C
Q03164_A
Q9UER7_A
P22681_A
Q03111_A
P23458_A
Q15788_A
P42345_C
P01116_A
P01112_A
Q8ND56_A
P30153_A
P15923_A
Q8NEZ4_A
P37231_B
P10276_B
P10276_A
P04637_A
Q15059_A
O15151_A
O75533_C
P43405_A
P00519_A
P06239_A
P06239_L
Q12923_A
O15169_A
Q14790_A
P06400_B
P03372_A
O96017_A
P27986_A
P06400_A
P31751_A
P41182_A
P38398_A
Q00987_M
P37231_X
Q00987_A
P08575_A
P55196_A
P11309_B
P62258_A
P29375_A
Q14839_A
P46100_A
Q9HD26_A
O60885_B
O60885_A
P21802_A
P52948_A
P11309_A
P31749_A
O15164_B
P22681_B
P61073_A
Q9ULV8_A
Q13191_A
Q06124_A
Q00987_B
Q9UMX1_A
Q9BZ95_A
O43791_A
Q9Y6K1_A
P55197_B
P

In [33]:
pssm_cc_all = np.array(pssm_cc_all)
pssm_cc_all

array([[-3.44161031e+11, -3.44161582e+11, -3.44160506e+11, ...,
         1.25468084e+00, -2.58819341e+00, -1.75016186e+01],
       [ 1.14049371e-01, -1.17701575e-01, -6.23927926e-01, ...,
         2.77320121e-02,  1.24520707e+00, -1.44347379e+01],
       [-1.08669431e-01,  7.87464006e-02, -5.49264972e-01, ...,
         7.28527607e-03,  1.24961656e+00, -9.69976994e+00],
       ...,
       [-1.08669431e-01,  7.87464006e-02, -5.49264972e-01, ...,
         7.28527607e-03,  1.24961656e+00, -8.91756135e+00],
       [ 1.91380292e+04,  2.58986863e+04,  3.08915688e+04, ...,
         9.44182558e+03,  2.72141556e+04,  5.94454653e+03],
       [-4.67494246e+09, -4.65691832e+09, -4.69212888e+09, ...,
         6.68415336e+03,  2.18344724e+04,  2.16116355e+03]])

In [34]:
pssm_cc_all_df = pd.DataFrame([x for x in pssm_cc_all])
pssm_cc_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
pssm_cc_all_df

,0,1,2,3,4,5,6,7,8,9,...,371,372,373,374,375,376,377,378,379,Uniprot_chain
0,-3.441610e+11,-3.441616e+11,-3.441605e+11,-3.435684e+11,-3.441614e+11,-3.441605e+11,-3.441607e+11,-3.441614e+11,-3.441610e+11,-3.441613e+11,...,1.009399,1.105459,1.104118,1.412884,1.061340,1.027220,1.254681,-2.588193,-17.501619,Q15796_A
1,1.140494e-01,-1.177016e-01,-6.239279e-01,-3.677876e-01,-1.856157e-02,1.270869e-01,-2.866042e-01,4.069974e-01,-6.367590e-01,-1.459437e-01,...,-0.418432,-0.133259,0.095190,-0.086131,-0.334669,-0.304776,0.027732,1.245207,-14.434738,P37231_D
2,-1.086694e-01,7.874640e-02,-5.492650e-01,-4.744483e-01,4.395664e-02,1.975553e-01,-3.385811e-01,2.938444e-01,-7.254583e-01,-1.442074e-01,...,-0.435966,-0.118482,0.074770,-0.061733,-0.345475,-0.363880,0.007285,1.249617,-9.699770,P04233_I
3,-1.241425e+11,-1.279210e+11,-1.258458e+11,-1.274210e+11,-1.279734e+11,-1.275330e+11,-1.276856e+11,-1.258457e+11,-1.275334e+11,-1.278473e+11,...,3553.042308,4749.531089,6402.086910,1600.080174,6712.647027,7095.558806,1253.023154,14748.459608,4006.015903,P35222_A
4,-3.144689e-02,4.491264e-02,-4.600888e-01,-3.971133e-01,3.540292e-02,2.268301e-01,-3.338560e-01,2.936799e-01,-7.659496e-01,-3.134123e-02,...,-0.429600,-0.080910,-0.055798,-0.003811,-0.324958,-0.337039,0.100761,-3.384587,-24.047767,Q01081_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,-1.086694e-01,7.874640e-02,-5.492650e-01,-4.744483e-01,4.395664e-02,1.975553e-01,-3.385811e-01,2.938444e-01,-7.254583e-01,-1.442074e-01,...,-0.435966,-0.118482,0.074770,-0.061733,-0.345475,-0.363880,0.007285,1.249617,-9.300997,O43791_B
129,-1.086694e-01,7.874640e-02,-5.492650e-01,-4.744483e-01,4.395664e-02,1.975553e-01,-3.385811e-01,2.938444e-01,-7.254583e-01,-1.442074e-01,...,-0.435966,-0.118482,0.074770,-0.061733,-0.345475,-0.363880,0.007285,1.249617,-7.031058,P04637_B
130,-1.086694e-01,7.874640e-02,-5.492650e-01,-4.744483e-01,4.395664e-02,1.975553e-01,-3.385811e-01,2.938444e-01,-7.254583e-01,-1.442074e-01,...,-0.435966,-0.118482,0.074770,-0.061733,-0.345475,-0.363880,0.007285,1.249617,-8.917561,P19174_A
131,1.913803e+04,2.589869e+04,3.089157e+04,1.148755e+04,5.146754e+03,4.845476e+04,2.719739e+04,4.512672e+04,1.066504e+04,5.355515e+04,...,20305.817581,8207.336182,10399.875342,10182.051541,18779.196504,17142.277459,9441.825580,27214.155633,5944.546528,Q92793_J


In [35]:
pssm_cc_all_df.to_csv('pssm_complete_cc_cancer.csv',index=False)
# pssm_ac_all_df.to_csv('pssm_ac.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np

#Secondary Structure
# load predicted ss features for sequences in the dataset
def aa_ss_concat(aa,ss):
    if len(aa)!= len(ss):
        return 'string length error!'
    else:
        new_str = ''
        for i in range(len(aa)):
            concat_str = aa[i]+ss[i]+','
            new_str = new_str+concat_str
    final_str = new_str[:-1]
    return final_str


df_org = pd.read_csv('E:/example_data_feature/SCRATCH-1D_2.0/SCRATCH-1D.ss',sep=',',header = None) #the generated file by SCRATCH1D SSPro
df_org.columns = ['col_1']


ss_idx=[]
seq_idx=[]
seq_idx = [2*x for x in list(range(int(df_org.shape[0]/2)))]
ss_idx = [x+1 for x in seq_idx]

# subset sequence dataframe and sse dataframe
df_seq = df_org.iloc[seq_idx]
df_seq.columns = ['seq_id']
df_ss = df_org.iloc[ss_idx]
df_ss.columns = ['seq_ss']

df_seq = df_seq.reset_index(drop=True)
df_ss = df_ss.reset_index(drop=True)

# join sequence & sse together
df_seq_ss = pd.merge(df_seq, df_ss,left_index=True, right_index=True)

# load id mapping file
df_id = pd.read_csv('seq_data.fasta',sep='#',header = None) #the input asta file used for SCRATCH1D SSPro
df_id.columns = ['col_1']

ss_idx=[]
seq_idx=[]
seq_idx = [2*x for x in list(range(int(df_id.shape[0]/2)))]
ss_idx = [x+1 for x in seq_idx]

# subset sequence dataframe and sse dataframe
df_seq = df_id.iloc[seq_idx]
df_seq.columns = ['seq_id']
df_ss = df_id.iloc[ss_idx]
df_ss.columns = ['seq']

df_seq = df_seq.reset_index(drop=True)
df_ss = df_ss.reset_index(drop=True)

# join sequence &  sse together
df_idx = pd.merge(df_seq, df_ss,left_index=True, right_index=True)
df_output_ss = pd.merge(df_idx, df_seq_ss, left_on=['seq_id'], right_on=['seq_id'])
df_output_ss['concat_seq'] = df_output_ss.apply(lambda x: aa_ss_concat(x['seq'],x['seq_ss']),axis=1)
df_output_ss.to_csv(output_ss_filename, encoding = 'utf-8', index = False, sep = '\t') # 'output_ss_filename' is the name of the output tsv you like


In [ ]:
import pickle
with open('E:/example_data_feature/peptide_feature_dict', 'rb') as f: # value: (sequence_length, 20) without sigmoid
    protein_feat_dict = pickle.load(f,encoding='latin1')
protein_feat_dict    

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(data=protein_feat_dict, orient='index')
 
#    .to_csv('E:/example_data_feature/PSSM/output_pssm_dict.csv', header=False))

In [3]:
import numpy as np
import math
import pickle
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

sigmoid_array=np.vectorize(sigmoid)

def padding_sigmoid_pssm(x,N):
	x = sigmoid_array(x)
	print(x.shape)    
	padding_array = np.zeros([N,x.shape[1]])
	if x.shape[0]>=N: # sequence is longer than N
		padding_array[:N,:x.shape[1]] = x[:N,:]
	else:
		padding_array[:x.shape[0],:x.shape[1]] = x
	print(padding_array.shape)
	return padding_array

def padding_intrinsic_disorder(x,N):
	padding_array = np.zeros([N,x.shape[1]])
	if x.shape[0]>=N: # sequence is longer than N
		padding_array[:N,:x.shape[1]] = x[:N,:]
	else:
		padding_array[:x.shape[0],:x.shape[1]] = x
	return padding_array

# load raw dense features, the directory dense_feature_dict and proprocessing need to be created first.
# with open('E:/example_data_feature/PSSM/output_pssm_dict_cancer', 'rb') as f: # value: (sequence_length, 20) without sigmoid
# 	peptide_pssm_dict = pickle.load(f,encoding='latin1')

with open('E:/temp/output_intrisic_dict_cancer', 'rb') as f: # value: (sequence_length, 3): long, short, anchor
	peptide_intrinsic_dict = pickle.load(f,encoding='latin1')



In [4]:
peptide_intrinsic_dict

{'Q15796_A': array([[0.3872, 0.4501, 0.8781, 0.4501],
        [0.3392, 0.4604, 0.8074, 0.4604],
        [0.3566, 0.4711, 0.7644, 0.4711],
        ...,
        [0.3215, 0.3023, 0.8556, 0.3023],
        [0.2849, 0.3118, 0.8857, 0.3118],
        [0.3117, 0.306 , 0.928 , 0.306 ]]),
 'P37231_D': array([[0.662 , 0.7219, 0.9796, 0.7219],
        [0.7415, 0.7106, 0.9775, 0.7106],
        [0.8013, 0.7001, 0.9753, 0.7001],
        ...,
        [0.0194, 0.1986, 0.4918, 0.1986],
        [0.0327, 0.1942, 0.6219, 0.1942],
        [0.0198, 0.1912, 0.6557, 0.1912]]),
 'P04233_I': array([[0.7415, 0.765 , 0.9624, 0.765 ],
        [0.7672, 0.715 , 0.9423, 0.715 ],
        [0.7916, 0.6651, 0.9316, 0.6651],
        ...,
        [0.5419, 0.715 , 0.9141, 0.715 ],
        [0.6576, 0.7243, 0.9553, 0.7243],
        [0.6269, 0.7418, 0.9665, 0.7418]]),
 'P35222_A': array([[0.5017, 0.5974, 0.892 , 0.5974],
        [0.4725, 0.6033, 0.8391, 0.6033],
        [0.4685, 0.6105, 0.7772, 0.6105],
        ...,
        [0.4

In [22]:
import pickle
with open('E:/example_data_feature/PSSM/output_pssm_dict2', 'rb') as f: # value: (sequence_length, 20) without sigmoid
	peptide_pssm_dict = pickle.load(f,encoding='latin1')
with open('E:/example_data_feature/PSSM/output_pssm_dict_kurang', 'rb') as f: # value: (sequence_length, 20) without sigmoid
	peptide_pssm_dict2 = pickle.load(f,encoding='latin1')

In [37]:
wew = pd.DataFrame.from_dict(data=peptide_pssm_dict, orient='index', columns = ['pep_seq','matrix'])
wew2 = pd.DataFrame.from_dict(data=peptide_pssm_dict2, orient='index', columns = ['pep_seq','matrix'])
wew

,pep_seq,matrix
1acy,FHCVPRDLSWLDLEANMCLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1qgc,GAEDAQDDLVPSIQDDGSESGACKI,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1vrk,YNKRKRIHIGPGRAFYTTKNIIGC,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1xcq,RRKWQKTGHAVRAIGRLSSSX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2b1a,STNPKPQRKTKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLG,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
7wvw,MLFII,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7yxw,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7zez,MYFINILTL,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
8gch,PPSNPPPRPPAEARKKPS,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [38]:
wew3 = pd.concat([wew,wew2]).drop_duplicates('pep_seq')
wew3

,pep_seq,matrix
1acy,FHCVPRDLSWLDLEANMCLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1qgc,GAEDAQDDLVPSIQDDGSESGACKI,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1vrk,YNKRKRIHIGPGRAFYTTKNIIGC,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1xcq,RRKWQKTGHAVRAIGRLSSSX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2b1a,STNPKPQRKTKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLG,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...
7t6u_l,QKFTSWFX,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7tbh_a,LSRRPSYRKILNDLSSDAP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
7wui_l,ISFGILLDLSRTSLP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
8gch_c,GAW,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [40]:
wew4 = pep_pairs2[pep_pairs2['pep_seq'].isin(wew3['pep_seq'])][['pdb_chain','pep_seq']]
wew4

,pdb_chain,pep_seq
0,1dev_b,SQSPNPNNPAEYCSTIPPLQQAQASGALSSPPPTVMVPVGV
1,1fm6_e,CPSSHSSLTERHKILHRLLQEGSPS
2,1icf_b,NNKYWLVKNSWGEEWGMGGYVKMAKDRRNHCGIASAASYPTV
3,1jdh_b,LGANDELISFKDEGEQEEKSSENSSAERDLADVKSSLV
4,1jmt_b,KKKVRKYWDVPPPGFEHITPMQYKAMQA
...,...,...
399,7rns_b,KDSYVGDEA
400,7rov_e,AXPLYISYDPVCRA
401,7tbh_a,LSRRPSYRKILNDLSSDAP
402,7wgo_b,LTERHKILHRLLQEG


In [43]:
pssm_pep_cancer = pd.merge(wew4, wew3,  on="pep_seq", how='left')
pssm_pep_cancer

,pdb_chain,pep_seq,matrix
0,1dev_b,SQSPNPNNPAEYCSTIPPLQQAQASGALSSPPPTVMVPVGV,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
1,1fm6_e,CPSSHSSLTERHKILHRLLQEGSPS,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
2,1icf_b,NNKYWLVKNSWGEEWGMGGYVKMAKDRRNHCGIASAASYPTV,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
3,1jdh_b,LGANDELISFKDEGEQEEKSSENSSAERDLADVKSSLV,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
4,1jmt_b,KKKVRKYWDVPPPGFEHITPMQYKAMQA,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
...,...,...,...
399,7rns_b,KDSYVGDEA,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
400,7rov_e,AXPLYISYDPVCRA,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
401,7tbh_a,LSRRPSYRKILNDLSSDAP,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."
402,7wgo_b,LTERHKILHRLLQEG,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,..."


In [47]:
len(pssm_pep_cancer)

404

In [52]:
feature_peptide = []
m=0
n=0
seq_list = []
for i in range(len(pssm_pep_cancer)):
    feature_pssm = padding_sigmoid_pssm(pssm_pep_cancer['matrix'][i],50)
    feature_peptide.append(feature_pssm.flatten())
    seq_list.append(pssm_pep_cancer['pdb_chain'][i])
feature_peptide = np.array(feature_peptide)
feature_peptide

array([[0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ]])

In [53]:
feature_peptide_df = pd.DataFrame(feature_peptide)
feature_peptide_df['pdb_chain']=seq_list
feature_peptide_df.to_csv('pssm_peptide_cancer_padded.csv',index=False)

In [34]:
pssm_prot_cancer = pd.DataFrame.from_dict(data=prot_pssm_dict_all, orient='index',columns = ['pep_seq','matrix'])
pssm_prot_cancer['Uniprot_chain'] = pssm_prot_cancer.index
pssm_prot_cancer

,pep_seq,matrix,Uniprot_chain
Q15796_A,MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVK...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q15796_A
P37231_D,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P37231_D
P04233_I,MHRRRSRSCREDQKPVMDDQRDLISNNEQLPMLGRRPGAPESKCSR...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04233_I
P35222_A,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P35222_A
Q01081_A,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q01081_A
...,...,...,...
O43791_B,MSRVPSPPPPAEMSSGPVAESWCYTQIKVVKFSYMWTINNFSFCRE...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",O43791_B
P04637_B,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P04637_B
P19174_A,MAGAASPCANGCGPGAPSDAEVLHLCRSLEVGTVMTLFYSKKSQRP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",P19174_A
Q92793_J,MAENLLDGPPNPKRAKLSSPGFSANDSTDFGSLFDLENDLPDELIP...,"[[4, -1, -2, -2, 0, -1, -1, 0, -2, -1, -1, -1,...",Q92793_J


In [63]:
pssm_prot_cancer['matrix'][i]

array([[ 4, -1, -2, ..., -3, -2,  0],
       [-2, -1,  4, ..., -4, -3, -3],
       [ 0, -3, -3, ..., -2, -2, -1],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [35]:
feature_prot = []
m=0
n=0
seq_list = []
for i in range(len(pssm_prot_cancer)):
    feature_pssm = padding_sigmoid_pssm(pssm_prot_cancer['matrix'][i],pad_prot_len)
    feature_prot.append(feature_pssm.flatten())
    seq_list.append(pssm_prot_cancer['Uniprot_chain'][i])
feature_prot = np.array(feature_prot)
feature_prot

C:\Users\Aulia Fadli\AppData\Local\Temp\ipykernel_16480\417436911.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


(501, 20)
(1441, 20)
(171, 20)
(1441, 20)
(164, 20)
(1441, 20)
(353, 20)
(1441, 20)
(169, 20)
(1441, 20)
(652, 20)
(1441, 20)
(207, 20)
(1441, 20)
(381, 20)
(1441, 20)
(166, 20)
(1441, 20)
(166, 20)
(1441, 20)
(2355, 20)
(1441, 20)
(166, 20)
(1441, 20)
(678, 20)
(1441, 20)
(191, 20)
(1441, 20)
(164, 20)
(1441, 20)
(289, 20)
(1441, 20)
(177, 20)
(1441, 20)
(440, 20)
(1441, 20)
(190, 20)
(1441, 20)
(209, 20)
(1441, 20)
(319, 20)
(1441, 20)
(169, 20)
(1441, 20)
(162, 20)
(1441, 20)
(295, 20)
(1441, 20)
(164, 20)
(1441, 20)
(164, 20)
(1441, 20)
(319, 20)
(1441, 20)
(164, 20)
(1441, 20)
(210, 20)
(1441, 20)
(319, 20)
(1441, 20)
(164, 20)
(1441, 20)
(304, 20)
(1441, 20)
(164, 20)
(1441, 20)
(166, 20)
(1441, 20)
(141, 20)
(1441, 20)
(165, 20)
(1441, 20)
(168, 20)
(1441, 20)
(319, 20)
(1441, 20)
(166, 20)
(1441, 20)
(164, 20)
(1441, 20)
(263, 20)
(1441, 20)
(405, 20)
(1441, 20)
(221, 20)
(1441, 20)
(166, 20)
(1441, 20)
(319, 20)
(1441, 20)
(421, 20)
(1441, 20)
(169, 20)
(1441, 20)
(164, 20)
(1

array([[0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ],
       [0.98201379, 0.26894142, 0.11920292, ..., 0.        , 0.        ,
        0.        ]])

In [36]:
feature_prot.shape

(133, 28820)

In [37]:
feature_prot_df = pd.DataFrame(feature_prot)
feature_prot_df['Uniprot_chain']=seq_list
feature_prot_df.to_csv('pssm_protein_cancer_padded.csv',index=False)

In [38]:
feature_prot_df

,0,1,2,3,4,5,6,7,8,9,...,28811,28812,28813,28814,28815,28816,28817,28818,28819,Uniprot_chain
0,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q15796_A
1,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P37231_D
2,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P04233_I
3,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P35222_A
4,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q01081_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O43791_B
129,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P04637_B
130,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P19174_A
131,0.982014,0.268941,0.119203,0.119203,0.5,0.268941,0.268941,0.5,0.119203,0.268941,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q92793_J


In [6]:
seq_len = np.array([len(seq) for seq in pep_pairs2['prot_seq'].unique()])
seq_len.mean()


1050.1984126984128

In [9]:
feature_peptide = []
m=0
n=0
seq_list = []

seq_len = [len(seq) for seq in pep_pairs2['prot_seq'].unique()]
seq_len.sort()
pad_prot_len = seq_len[int(0.8*len(seq_len))-1]
for i in peptide_intrinsic_dict.keys():
#     feature_pssm = padding_sigmoid_pssm(peptide_pssm_dict[i],50)
    feature_ID = padding_intrinsic_disorder(peptide_intrinsic_dict[i],1441)
    feature_peptide.append( feature_ID.flatten())
    seq_list.append(i)
feature_peptide = np.array(feature_peptide)
feature_peptide

array([[3.872e-01, 4.501e-01, 8.781e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [6.620e-01, 7.219e-01, 9.796e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [7.415e-01, 7.650e-01, 9.624e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [4.766e-01, 3.498e-01, 7.869e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [8.343e-01, 9.445e-01, 9.760e-01, ..., 4.860e-02, 1.210e-02,
        4.860e-02],
       [4.220e-01, 9.758e-01, 9.009e-01, ..., 5.500e-03, 2.000e-04,
        5.500e-03]])

In [10]:
feature_peptide_df = pd.DataFrame(feature_peptide)
feature_peptide_df['Uniprot_chain']=seq_list
feature_peptide_df.to_csv('IntrinsicD_protein_cancer_fixed.csv',index=False)

In [ ]:
df_fasta['Sequence']=fasta_sequence_list
df_fasta

In [ ]:
df_feature_peptide = pd.DataFrame(feature_peptide)
df_feature_peptide['Sequence']=seq_list
df_feature_peptide

In [ ]:
df_feature_peptide_all = df_feature_peptide.merge(df_fasta, on='Sequence')
df_feature_peptide_all

In [ ]:
df_feature_peptide_all.drop_duplicates(subset = '0_y').to_csv('peptide_features_pssm_intrinsic.csv',index=False)

In [ ]:
import linecache
pep_list = ['1acy','1a1r','1a2c','1a2x','1a38','1abi','1ai1','1an1','1apm','1as4','1avf','1avg',
            '1aw8','1axc','1bbr','1bh8','1bhx','1bqp','1c5w','1c8o','1cdk','1cdl','1cdm','1cf4',
            '1cff','1ckk','1clv','1cmk','1cn3','1cqt','1csb','1d3d','1d5s','1dd3','1dd4','1dev',
            '1dit','1dm4','1dml','1dow','1dpj','1ds5','1dt7','1dva','1dzi','1e0a','1e79','1eb1',
            '1eba','1ebd','1ees','1ehk','1ept','1ezx','1f0c']
prot_pssm_dict_all={}
prot_pssm_dict={}
seq_frag = []
seq_dict2 = {}
protein_num = 0 ### NEED TO BE CHANGED TO the total number of protein sequences
for i in range(55):
    filename_pssm = pep_list[i]+'.pssm' # need to name each individual fasta and pssm file with the same prefix
    filename_fasta = pep_list[i]+'.fasta'
    prot_key = pep_list[i]
    pssm_line_list= []
    
    with open('E:/example_data_feature/PSSM/input/'+filename_fasta,'r') as f: # directory to store fasta files (single file of each protein)
        for line in f.readlines():
            prot_seq = line.strip()
    with open('E:/example_data_feature/PSSM/blast_results/'+filename_pssm,'r') as f:  # directory to store pssm files (single file of each protein)

        for line in f.readlines()[3:-6]:
            line_list = line.strip().split(' ')
            wew =[x for x in line_list if x!=''][0:4]
            print(wew)
            if(wew[3]!='-I'):
                seq_frag.append(wew[1])
            line_list = [x for x in line_list if x!=''][2:22]
            if (len(line_list)!=20 or line_list[0]=='-I'):
#                 print('Error line:')
#                 print(line_list)
                continue
            line_list = [int(x) for x in line_list]
#             print(line_list)
            pssm_line_list.append(line_list)
        pssm_array = np.array(pssm_line_list)
        print(pssm_array.shape)
        if pssm_array.shape[1]!=20:
            print('Error!')
            print(filename_pssm)
        else:
            prot_pssm_dict_all[prot_key] = (prot_seq,pssm_array)
            prot_pssm_dict[prot_seq]=pssm_array
        print('seq_frag', seq_frag)

In [ ]:
with open('E:/example_data_feature/peptide_dense_feature_dict', 'rb') as f: # value: (sequence_length, 20) without sigmoid
    wew = pickle.load(f,encoding='latin1')


In [ ]:
wew_df = pd.DataFrame(wew.items())

wew_df[1]

In [4]:
pep_pairs2.drop_duplicates('pdb_chain')[['pdb_chain','pep_seq']].to_csv('ID3.csv',index=False)

## PHYSICOCHEMICAL FEATURES

In [7]:
amino_acid_set = { "A": 1, "C": 2, "B": 3, "E": 4, "D": 5, "G": 6, 
				"F": 7, "I": 8, "H": 9, "K": 10, "M": 11, "L": 12, 
				"O": 13, "N": 14, "Q": 15, "P": 16, "S": 17, "R": 18, 
				"U": 19, "T": 20, "W": 21, 
				"V": 22, "Y": 23, "X": 24, 
				"Z": 25 } # consider non-standard residues

amino_acid_num = 25

ss_set = {"H": 1, "C": 2, "E": 3} # revise order, not necessary if training your own model
ss_number = 3

physicochemical_set={'A': 1, 'C': 3, 'B': 7, 'E': 5, 'D': 5, 'G': 2, 'F': 1, 
			'I': 1, 'H': 6, 'K': 6, 'M': 1, 'L': 1, 'O': 7, 'N': 4, 
			'Q': 4, 'P': 1, 'S': 4, 'R': 6, 'U': 7, 'T': 4, 'W': 2, 
			'V': 1, 'Y': 4, 'X': 7, 'Z': 7}

residue_list = list(amino_acid_set.keys())
ss_list = list(ss_set.keys())

In [8]:
new_key_list = []
for i in residue_list:
    for j in ss_list:
        str_1 = str(i)+str(j)
        new_key_list.append(str_1)

new_value_list = [x+1 for x in list(range(amino_acid_num*ss_number))]

seq_ss_set = dict(zip(new_key_list,new_value_list))
seq_ss_number = amino_acid_num*ss_number #75

pad_pep_len = 50 

def label_sequence(line, pad_prot_len, res_ind):
	X = np.zeros(pad_prot_len)

	for i, res in enumerate(line[:pad_prot_len]):
		X[i] = res_ind[res]

	return X



In [10]:
seq_len = np.array([len(seq) for seq in pep_pairs2['prot_seq'].unique()])
avg_prot = seq_len.mean()
seq_len2 = np.array([len(seq) for seq in pep_pairs2['pep_seq'].unique()])
avg_pep = seq_len2.mean()

In [15]:
avg_pep.round()

18.0

In [84]:
pep_pairs2[pep_pairs2['pdb_chain']==i].iloc[0,3]

'CPSSHSSLTERHKILHRLLQEGSPS'

In [16]:
physico_pep_arr = []
pdb_chain_arr = []
for i in pep_pairs2['pdb_chain'].unique():
    print(i)
    feature = label_sequence(pep_pairs2[pep_pairs2['pdb_chain']==i].iloc[0,3], int(avg_pep.round()), physicochemical_set)
    physico_pep_arr.append(feature)
    pdb_chain_arr.append(i)

1dev_b
1fm6_e
1icf_b
1jdh_b
1jmt_b
1jpw_d
1jsp_a
1l3e_a
1lm8_h
1lqb_d
1mk2_b
1mox_c
1rdt_e
1sgh_b
1t08_b
2ao6_b
2e4h_b
2gl7_c
2hfp_b
2hqh_e
2jxc_b
2k42_b
2k8f_b
2kje_b
2ktb_a
2l1l_a
2lxt_c
2mh0_a
2msr_a
2mv7_b
2mzd_b
2n4q_a
2rfd_c
2rfe_e
2wax_b
2xa0_c
2xjy_b
2xjz_i
3avr_b
3e2u_e
3kmg_b
3njp_c
3nmx_d
3o34_b
3si5_x
3u5n_c
3u5o_i
3u5p_i
3u85_b
4day_c
4jol_e
4lk9_b
4lpz_c
4lx2_b
4n3y_a
4n4f_c
4nf9_c
4pky_b
4qmg_f
4wb7_i
4zri_c
5anr_c
5b75_b
5b76_b
5f6k_d
5f6l_j
5fcg_c
5grq_c
5hky_b
5j9s_b
5jcy_b
5l04_b
5nwm_b
5vau_e
5vay_e
5wbh_w
5wha_b
5whb_b
5wlb_b
5wlb_c
5wpl_b
5wpm_b
5xco_b
6d94_b
6f9w_b
6i7q_h
6iur_c
6lsb_b
6m64_b
6mgn_b
6mlc_e
6oni_d
6pdz_c
6qg8_b
6zvq_b
7aos_c
7apo_c
7bwn_l
7co1_b
7jq8_b
7jyn_b
7len_c
7lwh_b
7mla_b
7ndx_b
7onb_f
1a81_b
1bbz_b
1bhf_i
1cwd_p
1cwe_b
1d5g_b
1emu_b
1f8h_b
1f9e_q
1fbv_b
1ff1_b
1gux_e
1gwq_c
1gwr_c
1gxc_b
1h9o_b
1l2i_c
1lcj_b
1lck_b
1lkk_b
1lkl_b
1n4m_c
1o6k_c
1o9k_p
1pcg_e
1pic_b
1r2b_c
1t08_c
1t15_b
1t29_b
1t2v_f
1t4f_p
1t5z_b
1t63_b
1t65_b
1wm0_y
1x27_i

In [17]:
physico_pep_arr

[array([4., 4., 4., 1., 4., 1., 4., 4., 1., 1., 5., 4., 3., 4., 4., 1., 1.,
        1.]),
 array([3., 1., 4., 4., 6., 4., 4., 1., 4., 5., 6., 6., 6., 1., 1., 6., 6.,
        1.]),
 array([4., 4., 6., 4., 2., 1., 1., 6., 4., 4., 2., 2., 5., 5., 2., 2., 1.,
        2.]),
 array([1., 2., 1., 4., 5., 5., 1., 1., 4., 1., 6., 5., 5., 2., 5., 4., 5.,
        5.]),
 array([6., 6., 6., 1., 6., 6., 4., 2., 5., 1., 1., 1., 1., 2., 1., 5., 6.,
        1.]),
 array([2., 4., 2., 2., 5., 5., 1., 2., 1., 4., 5., 5., 1., 1., 4., 1., 6.,
        5.]),
 array([4., 6., 1., 6., 4., 6., 6., 2., 4., 4., 4., 4., 6., 6., 6., 6., 1.,
        1.]),
 array([2., 4., 1., 5., 5., 4., 2., 1., 1., 4., 1., 4., 4., 4., 5., 3., 5.,
        1.]),
 array([5., 1., 5., 1., 5., 1., 1., 1., 1., 4., 1., 1., 1., 5., 5., 5., 1.,
        4.]),
 array([1., 1., 4., 4., 4., 5., 4., 5., 1., 5., 1., 5., 1., 1., 1., 1., 4.,
        1.]),
 array([4., 1., 4., 1., 4., 4., 1., 1., 5., 4., 3., 4., 4., 1., 1., 1., 1.,
        4.]),
 array([1.

In [18]:
physico_pep_arr = np.array(physico_pep_arr)
physico_pep_arr_df = pd.DataFrame([x for x in physico_pep_arr])
# pssm_cc_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
# pssm_cc_all_df
physico_pep_arr_df['pdb_chain']=np.array(pdb_chain_arr)

In [19]:
physico_pep_arr_df.to_csv('pep_physicochemical_18.csv',index=False)

In [8]:
seq_len = [len(seq) for seq in pep_pairs2['prot_seq'].unique()]
seq_len.sort()
pad_prot_len = seq_len[int(0.8*len(seq_len))-1]
pad_prot_len

1464

In [21]:
physico_pro_arr = []
uni_chain_arr = []
seq_len = [len(seq) for seq in pep_pairs2['prot_seq'].unique()]
seq_len.sort()
pad_prot_len = seq_len[int(0.8*len(seq_len))-1]
for i in pep_pairs2['Uniprot_chain'].unique():
    print(i)
    seq = pep_pairs2[pep_pairs2['Uniprot_chain']==i].iloc[0,5]
    print(seq)
    feature = label_sequence(seq, int(avg_prot.round()), physicochemical_set)
    print(feature)
    physico_pro_arr.append(feature)
    uni_chain_arr.append(i)

Q15796_A
MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVKSLVKKLKKTGRLDELEKAITTQNCNTKCVTIPSTCSEIWGLSTPNTIDQWDTTGLYSFSEQTRSLDGRLQVSHRKGLPHVIYCRLWRWPDLHSHHELKAIENCEYAFNLKKDEVCVNPYHYQRVETPVLPPVLVPRHTEILTELPPLDDYTHSIPENTNFPAGIEPQSNYIPETPPPGYISEDGETSDQQLNQSMDTGSPAELSPTTLSPVNHSLDLQPVTYSEPAFWCSIAYYELNQRVGETFHASQPSLTVDGFTDPSNSERFCLGLLSNVNRNATVEMTRRHIGRGVRLYYIGGEVFAECLSDSAIFVQSPNCNQRYGWHPATVCKIPPGCNLKIFNNQEFAALLAQSVNQGFEAVYQLTRMCTIRMSFVKGWGAEYRRQTVTSTPCWIELHLNGPLQWLDKVLTQMGSPSVRCSSMS
[1. 4. 4. ... 0. 0. 0.]
P37231_D
MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGISSVDLSVMEDHSHSFDIKPFTTVDFSSISTPHYEDIPFTRTDPVVADYKYDLKLQEYQSAIKVEPASPPYYSEKTQLYNKPHEEPSNSLMAIECRVCGDKASGFHYGVHACEGCKGFFRRTIRLKLIYDRCDLNCRIHKKSRNKCQYCRFQKCLAVGMSHNAIRFGRMPQAEKEKLLAEISSDIDQLNPESADLRALAKHLYDSYIKSFPLTKAKARAILTGKTTDKSPFVIYDMNSLMMGEDKIKFKHITPLQEQSKEVAIRIFQGCQFRSVEAVQEITEYAKSIPGFVNLDLNDQVTLLKYGVHEIIYTMLASLMNKDGVLISEGQGFMTREFLKSLRKPFGDFMEPKFEFAVKFNALELDDSDLAIFIAVIILSGDRPGLLNVKPIEDIQDNLLQALELQLKLNHPESSQLFAKLLQKMTDLRQIVTEHVQLLQVIKKTETD

In [7]:
pad_prot_len

1464

In [125]:
len(seq_len)

102

In [22]:
physico_pro_arr = np.array(physico_pro_arr)
physico_pro_arr_df = pd.DataFrame([x for x in physico_pro_arr])
# pssm_cc_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
# pssm_cc_all_df
physico_pro_arr_df['Uniprot_chain']=np.array(uni_chain_arr)

In [23]:
physico_pro_arr_df.to_csv('prot_physicochemical_avg.csv',index=False)

## Amino Acid Features

In [24]:
amino_pep_arr = []
pdb_chain_arr = []
for i in pep_pairs2['pdb_chain'].unique():
    print(i)
    feature = label_sequence(pep_pairs2[pep_pairs2['pdb_chain']==i].iloc[0,3], int(avg_pep.round()), amino_acid_set)
    amino_pep_arr.append(feature)
    pdb_chain_arr.append(i)

1dev_b
1fm6_e
1icf_b
1jdh_b
1jmt_b
1jpw_d
1jsp_a
1l3e_a
1lm8_h
1lqb_d
1mk2_b
1mox_c
1rdt_e
1sgh_b
1t08_b
2ao6_b
2e4h_b
2gl7_c
2hfp_b
2hqh_e
2jxc_b
2k42_b
2k8f_b
2kje_b
2ktb_a
2l1l_a
2lxt_c
2mh0_a
2msr_a
2mv7_b
2mzd_b
2n4q_a
2rfd_c
2rfe_e
2wax_b
2xa0_c
2xjy_b
2xjz_i
3avr_b
3e2u_e
3kmg_b
3njp_c
3nmx_d
3o34_b
3si5_x
3u5n_c
3u5o_i
3u5p_i
3u85_b
4day_c
4jol_e
4lk9_b
4lpz_c
4lx2_b
4n3y_a
4n4f_c
4nf9_c
4pky_b
4qmg_f
4wb7_i
4zri_c
5anr_c
5b75_b
5b76_b
5f6k_d
5f6l_j
5fcg_c
5grq_c
5hky_b
5j9s_b
5jcy_b
5l04_b
5nwm_b
5vau_e
5vay_e
5wbh_w
5wha_b
5whb_b
5wlb_b
5wlb_c
5wpl_b
5wpm_b
5xco_b
6d94_b
6f9w_b
6i7q_h
6iur_c
6lsb_b
6m64_b
6mgn_b
6mlc_e
6oni_d
6pdz_c
6qg8_b
6zvq_b
7aos_c
7apo_c
7bwn_l
7co1_b
7jq8_b
7jyn_b
7len_c
7lwh_b
7mla_b
7ndx_b
7onb_f
1a81_b
1bbz_b
1bhf_i
1cwd_p
1cwe_b
1d5g_b
1emu_b
1f8h_b
1f9e_q
1fbv_b
1ff1_b
1gux_e
1gwq_c
1gwr_c
1gxc_b
1h9o_b
1l2i_c
1lcj_b
1lck_b
1lkk_b
1lkl_b
1n4m_c
1o6k_c
1o9k_p
1pcg_e
1pic_b
1r2b_c
1t08_c
1t15_b
1t29_b
1t2v_f
1t4f_p
1t5z_b
1t63_b
1t65_b
1wm0_y
1x27_i

In [25]:
amino_pep_arr = np.array(amino_pep_arr)
amino_pep_arr_df = pd.DataFrame([x for x in amino_pep_arr])
# pssm_cc_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
# pssm_cc_all_df
amino_pep_arr_df['pdb_chain']=np.array(pdb_chain_arr)
amino_pep_arr_df.to_csv('pep_amino_18.csv',index = False)

In [26]:
amino_pro_arr = []
uni_chain_arr = []
seq_len = [len(seq) for seq in pep_pairs2['prot_seq'].unique()]
seq_len.sort()
pad_prot_len = seq_len[int(0.8*len(seq_len))-1]
for i in pep_pairs2['Uniprot_chain'].unique():
    print(i)
    seq = pep_pairs2[pep_pairs2['Uniprot_chain']==i].iloc[0,5]
    print(seq)
    feature = label_sequence(seq, int(avg_prot.round()), amino_acid_set)
    print(feature)
    amino_pro_arr.append(feature)
    uni_chain_arr.append(i)

Q15796_A
MSSILPFTPPVVKRLLGWKKSAGGSGGAGGGEQNGQEEKWCEKAVKSLVKKLKKTGRLDELEKAITTQNCNTKCVTIPSTCSEIWGLSTPNTIDQWDTTGLYSFSEQTRSLDGRLQVSHRKGLPHVIYCRLWRWPDLHSHHELKAIENCEYAFNLKKDEVCVNPYHYQRVETPVLPPVLVPRHTEILTELPPLDDYTHSIPENTNFPAGIEPQSNYIPETPPPGYISEDGETSDQQLNQSMDTGSPAELSPTTLSPVNHSLDLQPVTYSEPAFWCSIAYYELNQRVGETFHASQPSLTVDGFTDPSNSERFCLGLLSNVNRNATVEMTRRHIGRGVRLYYIGGEVFAECLSDSAIFVQSPNCNQRYGWHPATVCKIPPGCNLKIFNNQEFAALLAQSVNQGFEAVYQLTRMCTIRMSFVKGWGAEYRRQTVTSTPCWIELHLNGPLQWLDKVLTQMGSPSVRCSSMS
[11. 17. 17. ...  0.  0.  0.]
P37231_D
MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGISSVDLSVMEDHSHSFDIKPFTTVDFSSISTPHYEDIPFTRTDPVVADYKYDLKLQEYQSAIKVEPASPPYYSEKTQLYNKPHEEPSNSLMAIECRVCGDKASGFHYGVHACEGCKGFFRRTIRLKLIYDRCDLNCRIHKKSRNKCQYCRFQKCLAVGMSHNAIRFGRMPQAEKEKLLAEISSDIDQLNPESADLRALAKHLYDSYIKSFPLTKAKARAILTGKTTDKSPFVIYDMNSLMMGEDKIKFKHITPLQEQSKEVAIRIFQGCQFRSVEAVQEITEYAKSIPGFVNLDLNDQVTLLKYGVHEIIYTMLASLMNKDGVLISEGQGFMTREFLKSLRKPFGDFMEPKFEFAVKFNALELDDSDLAIFIAVIILSGDRPGLLNVKPIEDIQDNLLQALELQLKLNHPESSQLFAKLLQKMTDLRQIVTEHVQLLQVI

In [27]:
amino_pro_arr = np.array(amino_pro_arr)
amino_pro_arr_df = pd.DataFrame([x for x in amino_pro_arr])
# pssm_cc_all_df['Uniprot_chain']=pep_pairs2['Uniprot_chain'].unique()
# pssm_cc_all_df
amino_pro_arr_df['Uniprot_chain']=np.array(uni_chain_arr)
amino_pro_arr_df.to_csv('pro_amino_avg.csv',index=False)

In [87]:
ID_prot = pd.read_csv('IntrinsicD_protein_cancer.csv')
amino_prot = pd.read_csv('pro_amino.csv')
physico_prot = pd.read_csv('prot_physicochemical.csv')

In [46]:
print(ID_prot.shape)
print(amino_prot.shape)
print(physico_prot.shape)

(133, 201)
(133, 1442)
(133, 1442)


In [91]:
feature_prot_df.shape

(133, 28821)

In [102]:
name = [x for x in range(0,31902)]
name.append('Uniprot_chain')

## GENERATE FEATURE PSSM FOLLOWS NATURE PAPER

In [103]:
prot_features_all = pd.concat([ID_prot.iloc[:,:-1],feature_prot_df.iloc[:,:-1],amino_prot.iloc[:,:-1],physico_prot],axis=1)
prot_features_all.columns = name
prot_features_all.to_csv('prot_features_all_new.csv',index=False)


In [104]:
prot_features_all

,0,1,2,3,4,5,6,7,8,9,...,31893,31894,31895,31896,31897,31898,31899,31900,31901,Uniprot_chain
0,0.3872,0.4501,0.8781,0.4501,0.3392,0.4604,0.8074,0.4604,0.3566,0.4711,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q15796_A
1,0.6620,0.7219,0.9796,0.7219,0.7415,0.7106,0.9775,0.7106,0.8013,0.7001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P37231_D
2,0.7415,0.7650,0.9624,0.7650,0.7672,0.7150,0.9423,0.7150,0.7916,0.6651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P04233_I
3,0.5017,0.5974,0.8920,0.5974,0.4725,0.6033,0.8391,0.6033,0.4685,0.6105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P35222_A
4,0.0631,0.3008,0.6174,0.3008,0.1137,0.2800,0.5762,0.2800,0.1399,0.2695,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q01081_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.6906,0.3051,0.9534,0.3051,0.7250,0.2859,0.9348,0.2859,0.6576,0.2708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O43791_B
129,0.9807,0.8488,0.9935,0.8488,0.9849,0.8331,0.9904,0.8331,0.9875,0.8187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P04637_B
130,0.4766,0.3498,0.7869,0.3498,0.3321,0.3261,0.6293,0.3261,0.3667,0.3057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,P19174_A
131,0.8343,0.9445,0.9760,0.9445,0.8966,0.9330,0.9746,0.9330,0.8894,0.9208,...,4.0,1.0,5.0,4.0,1.0,6.0,1.0,1.0,6.0,Q92793_J


In [77]:
ID_pep[ID_pep['pdb_chain'].isin(pep_pairs2['pdb_chain'].unique())].to_csv('IntrinsicD_peptide_cancer.csv')

In [105]:
ID_pep = pd.read_csv('IntrinsicD_peptide_cancer.csv')
amino_pep = pd.read_csv('pep_amino.csv')
physico_pep = pd.read_csv('pep_physicochemical.csv')
pssm_pep = pd.read_csv("pssm_peptide_cancer_padded.csv")

In [106]:
print(ID_pep.shape)
print(amino_pep.shape)
print(physico_pep.shape)
print(pssm_pep.shape)

(404, 201)
(404, 51)
(404, 51)
(404, 1001)


In [113]:
pep_features_all = pd.concat([ID_pep.iloc[:,:-1],pssm_pep.iloc[:,:-1],amino_pep.iloc[:,:-1],physico_pep],axis=1)
pep_features_all.columns = name2
pep_features_all.to_csv('pep_features_all_new.csv',index=False)

In [114]:
pep_features_all

,0,1,2,3,4,5,6,7,8,9,...,1291,1292,1293,1294,1295,1296,1297,1298,1299,pdb_chain
0,0.7369,0.4871,0.9653,0.4871,0.7951,0.4875,0.9604,0.4875,0.7120,0.5058,...,0,0,0,0,0,0,0,0,0,1a81_b
1,0.5951,0.2313,0.9423,0.2313,0.5951,0.4221,0.9423,0.4221,0.5951,0.3800,...,0,0,0,0,0,0,0,0,0,1bbz_b
2,0.2328,0.2237,0.9982,0.2237,0.2328,0.2237,0.9982,0.2237,0.2328,0.2237,...,0,0,0,0,0,0,0,0,0,1bhf_i
3,0.9562,0.9998,1.0000,0.9998,0.9562,0.9999,1.0000,0.9999,0.9562,0.9999,...,0,0,0,0,0,0,0,0,0,1cwd_p
4,0.7916,0.9705,0.9994,0.9705,0.7916,0.9740,0.9994,0.9740,0.7916,0.9740,...,0,0,0,0,0,0,0,0,0,1cwe_b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.7209,0.8862,0.9831,0.8862,0.7209,0.8959,0.9831,0.8959,0.7209,0.9014,...,0,0,0,0,0,0,0,0,0,7rns_b
400,0.0240,0.0449,0.7079,0.0449,0.0174,0.0458,0.6219,0.0458,0.0263,0.0471,...,0,0,0,0,0,0,0,0,0,7rov_e
401,0.3983,0.5009,0.9009,0.5009,0.4619,0.5022,0.8746,0.5022,0.4979,0.5027,...,0,0,0,0,0,0,0,0,0,7tbh_a
402,0.2918,0.3940,0.8074,0.3940,0.2164,0.3857,0.7079,0.3857,0.2783,0.3801,...,0,0,0,0,0,0,0,0,0,7wgo_b


In [111]:
name2 = [x for x in range(0,1300)]
name2.append('pdb_chain')